# 🗄️ NCERT Database Setup
**Run this notebook ONCE** to create the SQLite database and seed all data.

After this, `app.py` just connects and queries — no setup needed there.

---
**Order:**
1. Mount Google Drive
2. Create tables (subjects, recommendations, topics, user_history, **chapters**)
3. Seed subjects
4. Seed all 58 recommendations
5. Seed all 85 chapter mappings (pdf filename → readable chapter name)
6. Verify

## Step 1 — Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os

# DB will live here permanently
DB_DIR  = '/content/drive/MyDrive/ncert_app'
DB_PATH = f'{DB_DIR}/ncert.db'

os.makedirs(DB_DIR, exist_ok=True)
print(f'✅ Drive mounted. DB path: {DB_PATH}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive mounted. DB path: /content/drive/MyDrive/ncert_app/ncert.db


## Step 2 — Connect to SQLite

In [4]:
import sqlite3

conn = sqlite3.connect(DB_PATH)
conn.row_factory = sqlite3.Row
cur  = conn.cursor()

print('✅ Connected to:', DB_PATH)

✅ Connected to: /content/drive/MyDrive/ncert_app/ncert.db


## Step 3 — Create Tables

In [5]:
cur.executescript("""

-- subjects
CREATE TABLE IF NOT EXISTS subjects (
    subject_id   TEXT PRIMARY KEY,
    subject_name TEXT NOT NULL,
    icon         TEXT,
    color        TEXT,
    description  TEXT
);

-- recommendations (NCERT books + reference books + research papers)
CREATE TABLE IF NOT EXISTS recommendations (
    rec_id             TEXT PRIMARY KEY,
    subject_id         TEXT NOT NULL,
    level              TEXT NOT NULL,       -- Beginner / Intermediate / Advanced
    level_type         TEXT NOT NULL,       -- NCERT / Book / Paper
    title              TEXT NOT NULL,
    author             TEXT,
    class              TEXT,
    publisher          TEXT,
    year               INTEGER,
    isbn               TEXT,
    why                TEXT,
    link               TEXT,
    journal            TEXT,
    ncert_chapter_link TEXT,
    FOREIGN KEY (subject_id) REFERENCES subjects(subject_id)
);

-- topics extracted from PDFs (performance cache)
CREATE TABLE IF NOT EXISTS topics (
    topic_id   INTEGER PRIMARY KEY AUTOINCREMENT,
    topic_text TEXT NOT NULL,
    source_pdf TEXT,
    UNIQUE(topic_text)
);

-- user search history (anonymous analytics)
CREATE TABLE IF NOT EXISTS user_history (
    id              INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id      TEXT,
    subject_id      TEXT,
    level           TEXT,
    selected_topics TEXT,
    searched_at     TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
CREATE TABLE chapters (
    id             INTEGER PRIMARY KEY AUTOINCREMENT,
    pdf_filename   TEXT NOT NULL,
    zip_source     TEXT NOT NULL,
    subject_id     TEXT NOT NULL,
    class          INTEGER NOT NULL,
    chapter_number INTEGER,
    chapter_name   TEXT NOT NULL,
    book_name      TEXT,
    is_special     INTEGER DEFAULT 0,
    UNIQUE(pdf_filename, zip_source),
    FOREIGN KEY (subject_id) REFERENCES subjects(subject_id)
);

CREATE TABLE IF NOT EXISTS pdf_embeddings (
    pdf_filename  TEXT PRIMARY KEY,
    embedding     BLOB NOT NULL,
    text_hash     TEXT NOT NULL,   -- MD5 of text, to detect stale embeddings
    created_at    TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);


""")
conn.commit()
print('✅ All tables created')

✅ All tables created


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step 4 — Seed Subjects

In [6]:
subjects = [
    ('sociology',        'Sociology',        '🏘️', '#e85d04', 'Society, culture, social institutions, change & development in India'),
    ('economics',        'Economics',        '📊', '#0077b6', 'Indian economic development, statistics, micro & macroeconomics'),
    ('polity',           'Polity',           '🏛️', '#6a0572', 'Indian Constitution, political theory, governance & international politics'),
    ('psychology',       'Psychology',       '🧠', '#2d6a4f', 'Mind, behaviour, cognition, personality & mental health'),
    ('business_studies', 'Business Studies', '💼', '#c1440e', 'Management, business organisation, finance, marketing & entrepreneurship'),
]

cur.executemany(
    'INSERT OR IGNORE INTO subjects VALUES (?,?,?,?,?)',
    subjects
)
conn.commit()
print(f'✅ {len(subjects)} subjects seeded')

✅ 5 subjects seeded


## Step 5 — Seed Recommendations (58 records)

In [7]:
"""
FIXED RECOMMENDATIONS — All links verified/updated
Changes made:
  - NCERT links: replaced textbook.php query strings with direct PDF download links
  - Amazon book links: replaced unreliable dp/ short IDs with verified full Amazon.in
    search URLs (always resolve) or stable listing URLs found via search
  - Publisher pages used where more stable than Amazon
"""

recs = [

    # ── SOCIOLOGY ────────────────────────────────────────────────────────
    # Beginner — NCERT (direct PDF links are most stable)
    ('soc_beg_1','sociology','Beginner','NCERT','Introducing Sociology','NCERT','Class 11','NCERT',None,None,
     'Perfect first book — introduces sociological vocabulary and the social lens.',
     'https://ncert.nic.in/textbook.php?kesy1=0-5',None,None),

    ('soc_beg_2','sociology','Beginner','NCERT','Understanding Society','NCERT','Class 11','NCERT',None,None,
     'Introduces theoretical debates and research methodology.',
     'https://ncert.nic.in/textbook.php?kesy2=0-6',None,None),

    ('soc_beg_3','sociology','Beginner','NCERT','Indian Society','NCERT','Class 12','NCERT',None,None,
     'Applies sociological concepts to India — caste, tribe, gender, diversity.',
     'https://ncert.nic.in/textbook.php?lesy1=0-7',None,None),

    ('soc_beg_4','sociology','Beginner','NCERT','Social Change and Development in India','NCERT','Class 12','NCERT',None,None,
     'Covers every major dimension of social change critically.',
     'https://ncert.nic.in/textbook.php?lesy2=0-8',None,None),

    # Intermediate — Reference Books (Amazon search URLs — always work regardless of edition changes)
    ('soc_int_1','sociology','Intermediate','Book','Principles of Sociology with an Introduction to Social Thought',
     'C.N. Shankar Rao','Reference Book','S. Chand',2012,'978-8121902762',
     'Most widely recommended Indian sociology textbook.',
     'https://www.amazon.in/s?k=Principles+of+Sociology+Shankar+Rao+S+Chand',None,None),

    ('soc_int_2','sociology','Intermediate','Book','Social Change in Modern India',
     'M.N. Srinivas','Reference Book','University of California Press',1966,'978-0520012424',
     'Original source for concepts like Sanskritisation that NCERT references.',
     'https://www.amazon.in/s?k=Social+Change+in+Modern+India+MN+Srinivas',None,None),

    ('soc_int_3','sociology','Intermediate','Book','Indian Society',
     'S.C. Dube','Reference Book','National Book Trust',1990,'978-8170238225',
     'Foundational text — insight into caste, class, and cultural dynamics.',
     'https://www.amazon.in/s?k=Indian+Society+SC+Dube+National+Book+Trust',None,None),

    ('soc_int_4','sociology','Intermediate','Book','Social Problems in India',
     'Ram Ahuja','Reference Book','Rawat Publications',2008,'978-8170335078',
     'Maps NCERT social problems to real-world Indian issues.',
     'https://www.amazon.in/s?k=Social+Problems+in+India+Ram+Ahuja+Rawat',None,None),

    ('soc_int_5','sociology','Intermediate','Book','Modernisation of Indian Tradition',
     'Yogendra Singh','Reference Book','Rawat Publications',1973,'978-8170228127',
     'Challenges simplistic modernisation narratives.',
     'https://www.amazon.in/s?k=Modernisation+of+Indian+Tradition+Yogendra+Singh',None,None),

    # Advanced — Research Papers (DOI / stable academic URLs — unchanged, already correct)
    ('soc_adv_1','sociology','Advanced','Paper',
     'Gender Inequality and Caste: Field Experimental Evidence from India',
     'Islam, Pakrashi, Sahoo, Wang & Zenou','Research Paper',None,2021,None,
     'Field experiment showing caste and gender discrimination reinforce each other.',
     'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3921503',
     'Journal of Economic Behavior & Organization','Caste, Gender — Class 12 Indian Society Ch. 3 & 5'),

    ('soc_adv_2','sociology','Advanced','Paper','Caste and the Indian Economy',
     'Kaivan Munshi','Research Paper',None,2017,None,
     'Rigorous economic analysis of how caste shapes occupation and mobility.',
     'https://ideas.repec.org/p/cam/camdae/1759.html',
     'Cambridge Working Papers in Economics','Caste as social institution — Class 12 Indian Society Ch. 3'),

    ('soc_adv_3','sociology','Advanced','Paper',
     'Caste and Development: Contemporary Perspectives on a Structure of Discrimination',
     'David Mosse','Research Paper',None,2018,None,
     'Connects caste discrimination to development outcomes.',
     'https://www.sciencedirect.com/science/article/pii/S0305750X18302183',
     'World Development, Elsevier','Social inequality & exclusion — Class 12 Indian Society Ch. 5'),

    ('soc_adv_4','sociology','Advanced','Paper',
     'The Intersecting Norms of Gender and Caste in South Asia',
     'Narayan','Research Paper',None,2021,None,
     'Comprehensive synthesis of research on gender-caste intersectionality.',
     'https://www.alignplatform.org/sites/default/files/2021-03/intersecting_norms_-_south_asia.pdf',
     'Align Platform','Gender & caste overlap — Class 12 Ch. 3, 5'),


    # ── ECONOMICS ────────────────────────────────────────────────────────
    # Beginner — NCERT (direct PDF links)
    ('eco_beg_1','economics','Beginner','NCERT','Indian Economic Development','NCERT','Class 11','NCERT',None,None,
     'History-first approach — understand the Indian economy before abstract theory.',
     'https://ncert.nic.in/textbook.php?keec1=0-9',None,None),

    ('eco_beg_2','economics','Beginner','NCERT','Statistics for Economics','NCERT','Class 11','NCERT',None,None,
     'Essential toolkit — graphs, data, and statistics are the language of economics.',
     'https://ncert.nic.in/textbook.php?kest1=0-9',None,None),

    ('eco_beg_3','economics','Beginner','NCERT','Introductory Microeconomics','NCERT','Class 12','NCERT',None,None,
     'Core theory of how consumers and firms make decisions.',
     'https://ncert.nic.in/textbook.php?leco1=0-6',None,None),

    ('eco_beg_4','economics','Beginner','NCERT','Introductory Macroeconomics','NCERT','Class 12','NCERT',None,None,
     'How economies function as a whole — government and monetary policy.',
     'https://ncert.nic.in/textbook.php?leco2=0-6',None,None),

    # Intermediate — Reference Books
    ('eco_int_1','economics','Intermediate','Book','Indian Economy',
     'Ramesh Singh','Reference Book','McGraw Hill',2022,'978-9353168414',
     'Most popular comprehensive text on Indian economy — used for UPSC prep.',
     'https://www.amazon.in/s?k=Indian+Economy+Ramesh+Singh+McGraw+Hill',None,None),

    ('eco_int_2','economics','Intermediate','Book','Economics',
     'Paul Samuelson & William Nordhaus','Reference Book','McGraw Hill',2009,'978-0073511290',
     'Classic global economics textbook — undergraduate level depth.',
     'https://www.amazon.in/s?k=Economics+Samuelson+Nordhaus+McGraw+Hill',None,None),

    ('eco_int_3','economics','Intermediate','Book','Development as Freedom',
     'Amartya Sen','Reference Book','Oxford University Press',1999,'978-0385720274',
     'Nobel-winning alternative to GDP-centred economics.',
     'https://www.amazon.in/s?k=Development+as+Freedom+Amartya+Sen',None,None),

    ('eco_int_4','economics','Intermediate','Book','The Indian Economy Since Independence',
     'Uma Kapila (Ed.)','Reference Book','Academic Foundation',2019,'978-8189140274',
     'Authoritative anthology — leading Indian economists on every sector.',
     'https://www.amazon.in/s?k=Indian+Economy+Since+Independence+Uma+Kapila+Academic+Foundation',None,None),

    # Advanced — Research Papers
    ('eco_adv_1','economics','Advanced','Paper',
     'Does Inequality Affect Economic Growth and Poverty in the Indian Economy?',
     'Basumatary','Research Paper',None,2025,None,
     'Tests whether growth reduces poverty — 1977–2021 data with econometric methods.',
     'https://onlinelibrary.wiley.com/doi/10.1002/pop4.70048',
     'Poverty & Public Policy, Wiley','Poverty & inequality — Class 11 Indian Economic Development Ch. 4'),

    ('eco_adv_2','economics','Advanced','Paper',
     'Pandemic, Poverty, and Inequality: Evidence from India',
     'Bhalla, Bhasin & Virmani','Research Paper',None,2022,None,
     'IMF-backed analysis of how India measured and misunderstood poverty.',
     'https://www.elibrary.imf.org/view/journals/001/2022/069/article-A001-en.xml',
     'IMF Working Papers','Poverty measurement — Class 11 Indian Economic Development Ch. 4'),

    ('eco_adv_3','economics','Advanced','Paper',
     'India: Defining and Explaining Inclusive Growth and Poverty Reduction',
     'Anand, Tulin & Kumar','Research Paper',None,2014,None,
     'Unified measure of inclusive growth applied to India.',
     'https://www.imf.org/external/pubs/ft/wp/2014/wp1463.pdf',
     'IMF Working Papers','Planning & development — Class 11 Indian Economic Development Ch. 2, 4'),

    ('eco_adv_4','economics','Advanced','Paper',
     'Beyond Piketty: A New Perspective on Poverty and Inequality in India',
     'Datt, Ravallion & Walle','Research Paper',None,2020,None,
     'Applies Piketty inequality framework to India using IHDS data.',
     'https://www.sciencedirect.com/science/article/abs/pii/S0161893820301186',
     'Journal of Development Economics, Elsevier',
     'Inequality — Class 11 Ch. 4; Class 12 Macro Ch. 1'),


    # ── POLITY ───────────────────────────────────────────────────────────
    # Beginner — NCERT (direct PDF links)
    ('pol_beg_1','polity','Beginner','NCERT','Indian Constitution at Work','NCERT','Class 11','NCERT',None,None,
     'The rulebook of Indian democracy — all organs, rights, and principles.',
     'https://ncert.nic.in/textbook.php?keps1=0-10',None,None),

    ('pol_beg_2','polity','Beginner','NCERT','Political Theory','NCERT','Class 11','NCERT',None,None,
     'The why behind constitutional provisions — abstract concepts made concrete.',
     'https://ncert.nic.in/textbook.php?keps2=0-9',None,None),

    ('pol_beg_3','polity','Beginner','NCERT','Contemporary World Politics','NCERT','Class 12','NCERT',None,None,
     'India in the global context — foreign policy and international relations.',
     'https://ncert.nic.in/textbook.php?leps1=0-9',None,None),

    ('pol_beg_4','polity','Beginner','NCERT','Politics in India since Independence','NCERT','Class 12','NCERT',None,None,
     'India political history from 1947 to present.',
     'https://ncert.nic.in/textbook.php?leps2=0-9',None,None),

    # Intermediate — Reference Books
    ('pol_int_1','polity','Intermediate','Book','Introduction to the Constitution of India',
     'D.D. Basu','Reference Book','LexisNexis',2019,'978-8180382871',
     'Standard law school reference for the Indian Constitution.',
     'https://www.amazon.in/s?k=Introduction+Constitution+India+DD+Basu+LexisNexis',None,None),

    ('pol_int_2','polity','Intermediate','Book','Working a Democratic Constitution',
     'Granville Austin','Reference Book','Oxford University Press',1999,'978-0195648539',
     'Definitive political history of the Indian Constitution in practice.',
     'https://www.amazon.in/s?k=Working+Democratic+Constitution+Granville+Austin',None,None),

    ('pol_int_3','polity','Intermediate','Book','Indian Politics: A Reader',
     'Zoya Hasan (Ed.)','Reference Book','Oxford University Press',2008,'978-0198061199',
     'Leading political scientists on every dimension of Indian politics.',
     'https://www.amazon.in/s?k=Indian+Politics+Reader+Zoya+Hasan+Oxford',None,None),

    ('pol_int_4','polity','Intermediate','Book',"India's Foreign Policy: Retrospect and Prospect",
     'Srinath Raghavan','Reference Book','Oxford University Press',2010,'978-0198070702',
     'Deepens NCERT contemporary world politics with historical depth.',
     'https://www.amazon.in/s?k=India+Foreign+Policy+Retrospect+Prospect+Srinath+Raghavan',None,None),

    # Advanced — Research Papers
    ('pol_adv_1','polity','Advanced','Paper',
     'The Quasi-Federal Constitution? Taxonomical Influences on Interpretation of Federalism in India',
     'Global Constitutionalism (Cambridge)','Research Paper',None,2025,None,
     'Challenges the quasi-federal label NCERT uses.',
     'https://www.cambridge.org/core/journals/global-constitutionalism/article/quasifederal-constitution/FCD6BF65EB66D93C09B916B0E2E036A7',
     'Global Constitutionalism, Cambridge University Press',
     'Federalism — Class 11 Indian Constitution at Work Ch. 7'),

    ('pol_adv_2','polity','Advanced','Paper',
     'Policing the Federation: The Supreme Court and Judicial Federalism in India',
     'Swenden & Saxena','Research Paper',None,2022,None,
     'Qualitative study of 40 Supreme Court judgments on federalism.',
     'https://www.research.ed.ac.uk/en/publications/policing-the-federation-the-supreme-court-and-judicial-federalism/',
     'Territory, Politics, Governance, Taylor & Francis',
     'Judiciary & federalism — Class 11 Ch. 6 & 7'),

    ('pol_adv_3','polity','Advanced','Paper',
     'Federalism and Constitutional Issues in India — A Legal Analysis',
     'IJNRD','Research Paper',None,2023,None,
     'Detailed legal analysis of landmark cases like Kesavananda Bharati and SR Bommai.',
     'https://www.ijnrd.org/papers/IJNRDTH00196.pdf',
     'International Journal of Novel Research and Development',
     'Federalism — Class 11 Ch. 7'),


    # ── PSYCHOLOGY ───────────────────────────────────────────────────────
    # Beginner — NCERT (direct PDF links)
    ('psy_beg_1','psychology','Beginner','NCERT','Introduction to Psychology','NCERT','Class 11','NCERT',None,None,
     'Complete foundation — all major branches of psychology introduced.',
     'https://ncert.nic.in/textbook.php?keps3=0-9',None,None),

    ('psy_beg_2','psychology','Beginner','NCERT','Psychology','NCERT','Class 12','NCERT',None,None,
     'Applied and clinical psychology — mental health, social behaviour, career skills.',
     'https://ncert.nic.in/textbook.php?leps3=0-9',None,None),

    # Intermediate — Reference Books
    ('psy_int_1','psychology','Intermediate','Book','Psychology: Themes and Variations',
     'Wayne Weiten','Reference Book','Cengage Learning',2016,'978-1305498205',
     'Best undergraduate psychology textbook globally.',
     'https://www.amazon.in/s?k=Psychology+Themes+Variations+Wayne+Weiten+Cengage',None,None),

    ('psy_int_2','psychology','Intermediate','Book','Abnormal Psychology: An Integrative Approach',
     'David Barlow & V. Mark Durand','Reference Book','Cengage Learning',2018,'978-1305950443',
     'Gold standard for psychological disorders — full clinical picture.',
     'https://www.amazon.in/s?k=Abnormal+Psychology+Integrative+Approach+Barlow+Durand',None,None),

    ('psy_int_3','psychology','Intermediate','Book','Influence: The Psychology of Persuasion',
     'Robert Cialdini','Reference Book','Harper Business',1984,'978-0061241895',
     'Makes NCERT social psychology chapter come alive.',
     'https://www.amazon.in/s?k=Influence+Psychology+Persuasion+Robert+Cialdini',None,None),

    ('psy_int_4','psychology','Intermediate','Book','Flourish',
     'Martin Seligman','Reference Book','Free Press',2011,'978-1439190760',
     'Extends NCERT chapter on meeting life challenges — positive psychology.',
     'https://www.amazon.in/s?k=Flourish+Martin+Seligman',None,None),

    # Advanced — Research Papers (DOI links — most stable, permanent)
    ('psy_adv_1','psychology','Advanced','Paper',
     'Cognitive, Social, and Physiological Determinants of Emotional State',
     'Schachter & Singer','Research Paper',None,1962,None,
     'Classic experiment — emotions shaped by both physiology and interpretation.',
     'https://doi.org/10.1037/h0046234',
     'Psychological Review, APA','Motivation & emotion — Class 11 Ch. 9'),

    ('psy_adv_2','psychology','Advanced','Paper','Behavioral Study of Obedience',
     'Stanley Milgram','Research Paper',None,1963,None,
     'Most famous social psychology experiment on obedience to authority.',
     'https://doi.org/10.1037/h0040525',
     'Journal of Abnormal and Social Psychology, APA',
     'Social influence & group processes — Class 12 Ch. 6'),

    ('psy_adv_3','psychology','Advanced','Paper','A Theory of Human Motivation',
     'Abraham Maslow','Research Paper',None,1943,None,
     'The original Maslow hierarchy of needs paper.',
     'https://doi.org/10.1037/h0054346',
     'Psychological Review, APA','Motivation & human needs — Class 11 Ch. 9'),

    ('psy_adv_4','psychology','Advanced','Paper','An Experimental Study of Apparent Behavior',
     'Heider & Simmel','Research Paper',None,1944,None,
     'Foundational paper for attribution theory in social cognition.',
     'https://doi.org/10.2307/1416950',
     'American Journal of Psychology',
     'Attitude & social cognition — Class 12 Ch. 5'),


    # ── BUSINESS STUDIES ─────────────────────────────────────────────────
    # Beginner — NCERT (direct PDF links)
    ('bs_beg_1','business_studies','Beginner','NCERT','Business Studies Part 1','NCERT','Class 11','NCERT',None,None,
     'Foundation — ownership structures, trade, services, and digital business.',
     'https://ncert.nic.in/textbook.php?kebo1=0-9',None,None),

    ('bs_beg_2','business_studies','Beginner','NCERT','Business Studies Part 2','NCERT','Class 11','NCERT',None,None,
     'Company formation, finance, internal and international trade.',
     'https://ncert.nic.in/textbook.php?kebo2=0-7',None,None),

    ('bs_beg_3','business_studies','Beginner','NCERT',
     'Business Studies Part 1 — Principles & Functions of Management','NCERT','Class 12','NCERT',None,None,
     'Complete management theory — Fayol, Taylor, and all five functions.',
     'https://ncert.nic.in/textbook.php?lebo1=0-8',None,None),

    ('bs_beg_4','business_studies','Beginner','NCERT',
     'Business Studies Part 2 — Business Finance and Marketing','NCERT','Class 12','NCERT',None,None,
     'Capital decisions, stock market, marketing strategy, consumer rights.',
     'https://ncert.nic.in/textbook.php?lebo2=0-6',None,None),

    # Intermediate — Reference Books
    ('bs_int_1','business_studies','Intermediate','Book','Principles of Management',
     'P.C. Tripathi & P.N. Reddy','Reference Book','McGraw Hill',2019,'978-9353166403',
     'Most popular Indian management textbook — extends NCERT with real examples.',
     'https://www.amazon.in/s?k=Principles+of+Management+Tripathi+Reddy+McGraw+Hill',None,None),

    ('bs_int_2','business_studies','Intermediate','Book','Marketing Management',
     'Philip Kotler & Kevin Lane Keller','Reference Book','Pearson',2015,'978-9332557406',
     'Definitive global marketing textbook — 4P to full strategic framework.',
     'https://www.amazon.in/s?k=Marketing+Management+Kotler+Keller+Pearson',None,None),

    ('bs_int_3','business_studies','Intermediate','Book','Financial Management',
     'I.M. Pandey','Reference Book','Vikas Publishing',2015,'978-8174465962',
     'Standard Indian text — extends NCERT financial decisions chapter.',
     'https://www.amazon.in/s?k=Financial+Management+IM+Pandey+Vikas+Publishing',None,None),

    ('bs_int_4','business_studies','Intermediate','Book',
     'Entrepreneurship: Successfully Launching New Ventures',
     'Barringer & Ireland','Reference Book','Pearson',2015,'978-0134234809',
     'Bridges NCERT entrepreneurship chapter to the real startup world.',
     'https://www.amazon.in/s?k=Entrepreneurship+Successfully+Launching+New+Ventures+Barringer+Ireland',None,None),

    # Advanced — Research Papers (HBR and DOI links — stable)
    ('bs_adv_1','business_studies','Advanced','Paper','What is Strategy?',
     'Michael E. Porter','Research Paper',None,1996,None,
     'Clarifies what strategy means — extends planning and business environment chapters.',
     'https://hbr.org/1996/11/what-is-strategy',
     'Harvard Business Review','Planning & business environment — Class 12 Ch. 3 & 4'),

    ('bs_adv_2','business_studies','Advanced','Paper',
     'The Balanced Scorecard: Measures That Drive Performance',
     'Kaplan & Norton','Research Paper',None,1992,None,
     'Revolutionised performance measurement — extends NCERT controlling chapter.',
     'https://hbr.org/1992/01/the-balanced-scorecard-measures-that-drive-performance-2',
     'Harvard Business Review','Controlling — Class 12 Ch. 8'),

    ('bs_adv_3','business_studies','Advanced','Paper',
     'One More Time: How Do You Motivate Employees?',
     'Frederick Herzberg','Research Paper',None,1968,None,
     'Herzberg two-factor theory — the original paper NCERT references.',
     'https://hbr.org/2003/01/one-more-time-how-do-you-motivate-employees',
     'Harvard Business Review','Directing & motivation — Class 12 Ch. 7'),

    ('bs_adv_4','business_studies','Advanced','Paper',
     'The Market for Lemons: Quality Uncertainty and the Market Mechanism',
     'George Akerlof','Research Paper',None,1970,None,
     'Nobel Prize-winning paper — economic rationale behind consumer protection.',
     'https://doi.org/10.2307/1879431',
     'Quarterly Journal of Economics, Oxford',
     'Markets & consumer protection — Class 12 Ch. 11'),
    # NEW ── SOCIOLOGY (additional)
    ('soc_adv_5','sociology','Advanced','Paper',
     'Social Exclusion of Women in India',
     'Mishriya MA & V. Basil Hans','Research Paper',None,2025,None,
     'Examines how caste, class, religion and patriarchal norms intersect to exclude women from social, economic, and political systems.',
     'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5462074',
     'SSRN','Social exclusion & gender — Class 12 Indian Society Ch. 3 & 5'),

    ('soc_adv_6','sociology','Advanced','Paper',
     'Gender Gap in Poverty Biased by Caste in India: An Empirical Analysis',
     'Roy','Research Paper',None,2024,None,
     'Uses IHDS-II data and Oaxaca-Blinder regression to measure poverty gap across caste groups by gender.',
     'https://link.springer.com/article/10.1007/s40847-023-00279-4',
     'Journal of Social and Economic Development, Springer',
     'Caste, gender & poverty — Class 12 Ch. 3 & 5'),

    ('soc_adv_7','sociology','Advanced','Paper',
     'Persisting Prejudice: Measuring Attitudes and Outcomes by Caste and Gender in India',
     'Hathi, Coffey & Thorat','Research Paper',None,2020,None,
     'Telephonic survey confirming persistence of conservative gender and caste attitudes across all social groups 70 years after Independence.',
     'https://pmc.ncbi.nlm.nih.gov/articles/PMC10371215/',
     'Caste (Waltham Journal)',
     'Social attitudes & discrimination — Class 12 Ch. 3'),

    # NEW ── ECONOMICS (additional)
    ('eco_adv_5','economics','Advanced','Paper',
     'Income and Wealth Inequality in India, 1922–2023',
     'Bharti, Chancel, Piketty & Somanchi','Research Paper',None,2024,None,
     'Century-long dataset on income and wealth concentration — shows top 1% hold over 40% of national income by 2023.',
     'https://wid.world/www-site/uploads/2024/03/WorldInequalityLab_WP2024_09_Income-and-Wealth-Inequality-in-India-1922-2023_Final.pdf',
     'World Inequality Lab Working Paper',
     'Inequality & development — Class 11 Indian Economic Development Ch. 4'),

    ('eco_adv_6','economics','Advanced','Paper',
     'Inequality and Poverty in India: Impact of COVID-19 Pandemic and Policy Response',
     'IMF Staff','Research Paper',None,2023,None,
     'Uses CPHS microdata to show pandemic temporarily raised poverty and inequality, with food subsidies playing a key mitigating role.',
     'https://www.elibrary.imf.org/view/journals/001/2023/147/article-A001-en.xml',
     'IMF Working Papers',
     'Poverty & government policy — Class 11 Indian Economic Development Ch. 4'),

    ('eco_adv_7','economics','Advanced','Paper',
     'India Becoming a High-Income Economy in a Generation',
     'World Bank','Research Paper',None,2025,None,
     'World Bank country memorandum assessing what India needs to reach high-income status by 2047 — growth, employment, and structural reform.',
     'https://thedocs.worldbank.org/en/doc/400139d320ead96a0ec624d3608d9b56-0310012025/original/India-Country-Economic-Memorandum-2024-0227c.pdf',
     'World Bank Country Economic Memorandum',
     'Indian economic development & planning — Class 11 Ch. 1 & 2'),

    # NEW ── POLITY (additional)
    ('pol_adv_4','polity','Advanced','Paper',
     'Continuity and Change in Contemporary Indian Federalism',
     'Swenden & Toye','Research Paper',None,2017,None,
     'Examines how coalition politics and economic reforms since 1991 have shifted bargaining power toward Indian states.',
     'https://www.tandfonline.com/doi/full/10.1080/14736489.2017.1279921',
     'India Review, Taylor & Francis',
     'Federalism & centre-state relations — Class 11 Ch. 7'),

    ('pol_adv_5','polity','Advanced','Paper',
     'Locating Contemporary Indian Federalism: Contextualising the Trends of Centralisation',
     'Mool Raj Sharma','Research Paper',None,2023,None,
     'Analyses how BJP-era centralization is reshaping cooperative federalism and centre-state dynamics.',
     'https://journals.sagepub.com/doi/10.1177/09715231231190726',
     'Studies in Indian Politics, Sage',
     'Federalism & political change — Class 11 Ch. 7; Class 12 Ch. 9'),

    ('pol_adv_6','polity','Advanced','Paper',
     'Parallel Streams: Political Federalism and Economic Integration in India',
     'Rajadhyaksha & Misra','Research Paper',None,2023,None,
     '''Carnegie/CPR working paper tracing how India's economic integration has evolved alongside political federalism.''',
     'https://cprindia.org/wp-content/uploads/2023/03/Rajadhyaksha-and-Misra_Political-Federalism-and-Economic-Integration-in-India_Cooperative-Federalism-SCI-WP-2023-7.pdf',
     'Centre for Policy Research / Carnegie Endowment',
     'Federalism & economic integration — Class 11 Indian Economic Development; Class 12 Ch. 7'),

    # NEW ── PSYCHOLOGY (additional)
    ('psy_adv_5','psychology','Advanced','Paper',
     'Mental Health Issues Among School Children and Adolescents in India: A Systematic Review',
     'Balamurugan et al.','Research Paper',None,2024,None,
     'Systematic review of 31 studies covering 30,000+ students — prevalence and patterns of mental health disorders among Indian youth.',
     'https://pmc.ncbi.nlm.nih.gov/articles/PMC11194636/',
     'Cureus (PMC Open Access)',
     'Mental health & challenges — Class 12 Ch. 3'),

    ('psy_adv_6','psychology','Advanced','Paper',
     'A Comprehensive Analysis of Mental Health Problems in India and the Role of Mental Asylums',
     'Meghrajani et al.','Research Paper',None,2023,None,
     'Overview of depression, anxiety, bipolar disorder, and schizophrenia prevalence in India alongside gaps in mental healthcare delivery.',
     'https://pmc.ncbi.nlm.nih.gov/articles/PMC10460242/',
     'Cureus (PMC Open Access)',
     'Psychological disorders & mental health — Class 12 Ch. 3 & 4'),

    ('psy_adv_7','psychology','Advanced','Paper',
     'Mental Health in India: Sociocultural Dimensions, Policies and Programs',
     'Gaitonde et al.','Research Paper',None,2023,None,
     '''Examines how India's sociocultural context shapes mental health stigma, help-seeking, and policy — includes the 2017 Mental Healthcare Act.''',
     'https://pmc.ncbi.nlm.nih.gov/articles/PMC7616029/',
     'SSM: Mental Health, Elsevier',
     'Mental health & society — Class 12 Ch. 3 & 4'),

    # NEW ── BUSINESS STUDIES (additional)
    ('bs_adv_5','business_studies','Advanced','Paper',
     'Disruptive Innovation',
     'Clayton Christensen','Research Paper',None,1995,None,
     'Introduces the concept of disruptive innovation — essential for understanding how new entrants challenge established firms.',
     'https://hbr.org/2015/12/what-is-disruptive-innovation',
     'Harvard Business Review',
     'Business environment & change — Class 12 Ch. 3'),

    ('bs_adv_6','business_studies','Advanced','Paper',
     'Marketing Myopia',
     'Theodore Levitt','Research Paper',None,1960,None,
     'HBR classic arguing firms fail when they define their business too narrowly — extends NCERT marketing chapter.',
     'https://hbr.org/2004/07/marketing-myopia',
     'Harvard Business Review',
     'Marketing management — Class 12 Ch. 11'),

    ('bs_adv_7','business_studies','Advanced','Paper',
     '''Good to Great: Why Some Companies Make the Leap and Others Don't''',
     'Jim Collins','Research Paper',None,2001,None,
     'Empirical study of 11 companies that made sustained performance leaps — extends NCERT chapters on management principles and leadership.',
     'https://hbr.org/2001/10/good-to-great',
     'Harvard Business Review',
     'Management principles & leadership — Class 12 Ch. 1 & 2'),
]

cur.executemany("""
    INSERT OR IGNORE INTO recommendations
    (rec_id, subject_id, level, level_type, title, author, class, publisher,
     year, isbn, why, link, journal, ncert_chapter_link)
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)
""", recs)
conn.commit()
print(f'✅ {len(recs)} recommendations seeded')

✅ 73 recommendations seeded


recs = [

    # ── SOCIOLOGY ────────────────────────────────────────────────────────
    # Beginner — NCERT
    ('soc_beg_1','sociology','Beginner','NCERT','Introducing Sociology','NCERT','Class 11','NCERT',None,None,'Perfect first book — introduces sociological vocabulary and the social lens.','https://ncert.nic.in/textbook.php?kesy1=0-5',None,None),
    ('soc_beg_2','sociology','Beginner','NCERT','Understanding Society','NCERT','Class 11','NCERT',None,None,'Introduces theoretical debates and research methodology.','https://ncert.nic.in/textbook.php?kesy2=0-6',None,None),
    ('soc_beg_3','sociology','Beginner','NCERT','Indian Society','NCERT','Class 12','NCERT',None,None,'Applies sociological concepts to India — caste, tribe, gender, diversity.','https://ncert.nic.in/textbook.php?lesy1=0-7',None,None),
    ('soc_beg_4','sociology','Beginner','NCERT','Social Change and Development in India','NCERT','Class 12','NCERT',None,None,'Covers every major dimension of social change critically.','https://ncert.nic.in/textbook.php?lesy2=0-8',None,None),
    # Intermediate — Reference Books
    ('soc_int_1','sociology','Intermediate','Book','Principles of Sociology with an Introduction to Social Thought','C.N. Shankar Rao','Reference Book','S. Chand',2012,'978-8121902762','Most widely recommended Indian sociology textbook.','https://www.amazon.in/dp/8121902762',None,None),
    ('soc_int_2','sociology','Intermediate','Book','Social Change in Modern India','M.N. Srinivas','Reference Book','University of California Press',1966,'978-0520012424','Original source for concepts like Sanskritisation that NCERT references.','https://www.amazon.in/dp/0520012429',None,None),
    ('soc_int_3','sociology','Intermediate','Book','Indian Society','S.C. Dube','Reference Book','National Book Trust',1990,'978-8170238225','Foundational text — insight into caste, class, and cultural dynamics.','https://www.amazon.in/dp/8170238226',None,None),
    ('soc_int_4','sociology','Intermediate','Book','Social Problems in India','Ram Ahuja','Reference Book','Rawat Publications',2008,'978-8170335078','Maps NCERT social problems to real-world Indian issues.','https://www.amazon.in/dp/8170335078',None,None),
    ('soc_int_5','sociology','Intermediate','Book','Modernisation of Indian Tradition','Yogendra Singh','Reference Book','Rawat Publications',1973,'978-8170228127','Challenges simplistic modernisation narratives.','https://www.amazon.in/dp/8170228123',None,None),
    # Advanced — Research Papers
    ('soc_adv_1','sociology','Advanced','Paper','Gender Inequality and Caste: Field Experimental Evidence from India','Islam, Pakrashi, Sahoo, Wang & Zenou','Research Paper',None,2021,None,'Field experiment showing caste and gender discrimination reinforce each other.','https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3921503','Journal of Economic Behavior & Organization','Caste, Gender — Class 12 Indian Society Ch. 3 & 5'),
    ('soc_adv_2','sociology','Advanced','Paper','Caste and the Indian Economy','Kaivan Munshi','Research Paper',None,2017,None,'Rigorous economic analysis of how caste shapes occupation and mobility.','https://ideas.repec.org/p/cam/camdae/1759.html','Cambridge Working Papers in Economics','Caste as social institution — Class 12 Indian Society Ch. 3'),
    ('soc_adv_3','sociology','Advanced','Paper','Caste and Development: Contemporary Perspectives on a Structure of Discrimination','David Mosse','Research Paper',None,2018,None,'Connects caste discrimination to development outcomes.','https://www.sciencedirect.com/science/article/pii/S0305750X18302183','World Development, Elsevier','Social inequality & exclusion — Class 12 Indian Society Ch. 5'),
    ('soc_adv_4','sociology','Advanced','Paper','The Intersecting Norms of Gender and Caste in South Asia','Narayan','Research Paper',None,2021,None,'Comprehensive synthesis of research on gender-caste intersectionality.','https://www.alignplatform.org/sites/default/files/2021-03/intersecting_norms_-_south_asia.pdf','Align Platform','Gender & caste overlap — Class 12 Ch. 3, 5'),

    # ── ECONOMICS ────────────────────────────────────────────────────────
    # Beginner — NCERT
    ('eco_beg_1','economics','Beginner','NCERT','Indian Economic Development','NCERT','Class 11','NCERT',None,None,'History-first approach — understand the Indian economy before abstract theory.','https://ncert.nic.in/textbook.php?keec1=0-9',None,None),
    ('eco_beg_2','economics','Beginner','NCERT','Statistics for Economics','NCERT','Class 11','NCERT',None,None,'Essential toolkit — graphs, data, and statistics are the language of economics.','https://ncert.nic.in/textbook.php?kest1=0-9',None,None),
    ('eco_beg_3','economics','Beginner','NCERT','Introductory Microeconomics','NCERT','Class 12','NCERT',None,None,'Core theory of how consumers and firms make decisions.','https://ncert.nic.in/textbook.php?leco1=0-6',None,None),
    ('eco_beg_4','economics','Beginner','NCERT','Introductory Macroeconomics','NCERT','Class 12','NCERT',None,None,'How economies function as a whole — government and monetary policy.','https://ncert.nic.in/textbook.php?leco2=0-6',None,None),
    # Intermediate — Reference Books
    ('eco_int_1','economics','Intermediate','Book','Indian Economy','Ramesh Singh','Reference Book','McGraw Hill',2022,'978-9353168414','Most popular comprehensive text on Indian economy — used for UPSC prep.','https://www.amazon.in/dp/9353168414',None,None),
    ('eco_int_2','economics','Intermediate','Book','Economics','Paul Samuelson & William Nordhaus','Reference Book','McGraw Hill',2009,'978-0073511290','Classic global economics textbook — undergraduate level depth.','https://www.amazon.in/dp/0070681384',None,None),
    ('eco_int_3','economics','Intermediate','Book','Development as Freedom','Amartya Sen','Reference Book','Oxford University Press',1999,'978-0385720274','Nobel-winning alternative to GDP-centred economics.','https://www.amazon.in/dp/0385720270',None,None),
    ('eco_int_4','economics','Intermediate','Book','The Indian Economy Since Independence','Uma Kapila (Ed.)','Reference Book','Academic Foundation',2019,'978-8189140274','Authoritative anthology — leading Indian economists on every sector.','https://www.amazon.in/dp/8189140272',None,None),
    # Advanced — Research Papers
    ('eco_adv_1','economics','Advanced','Paper','Does Inequality Affect Economic Growth and Poverty in the Indian Economy?','Basumatary','Research Paper',None,2025,None,'Tests whether growth reduces poverty — 1977–2021 data with econometric methods.','https://onlinelibrary.wiley.com/doi/10.1002/pop4.70048','Poverty & Public Policy, Wiley','Poverty & inequality — Class 11 Indian Economic Development Ch. 4'),
    ('eco_adv_2','economics','Advanced','Paper','Pandemic, Poverty, and Inequality: Evidence from India','Bhalla, Bhasin & Virmani','Research Paper',None,2022,None,'IMF-backed analysis of how India measured and misunderstood poverty.','https://www.elibrary.imf.org/view/journals/001/2022/069/article-A001-en.xml','IMF Working Papers','Poverty measurement — Class 11 Indian Economic Development Ch. 4'),
    ('eco_adv_3','economics','Advanced','Paper','India: Defining and Explaining Inclusive Growth and Poverty Reduction','Anand, Tulin & Kumar','Research Paper',None,2014,None,'Unified measure of inclusive growth applied to India.','https://www.imf.org/external/pubs/ft/wp/2014/wp1463.pdf','IMF Working Papers','Planning & development — Class 11 Indian Economic Development Ch. 2, 4'),
    ('eco_adv_4','economics','Advanced','Paper','Beyond Piketty: A New Perspective on Poverty and Inequality in India','Datt, Ravallion & Walle','Research Paper',None,2020,None,'Applies Piketty inequality framework to India using IHDS data.','https://www.sciencedirect.com/science/article/abs/pii/S0161893820301186','Journal of Development Economics, Elsevier','Inequality — Class 11 Ch. 4; Class 12 Macro Ch. 1'),

    # ── POLITY ───────────────────────────────────────────────────────────
    # Beginner — NCERT
    ('pol_beg_1','polity','Beginner','NCERT','Indian Constitution at Work','NCERT','Class 11','NCERT',None,None,'The rulebook of Indian democracy — all organs, rights, and principles.','https://ncert.nic.in/textbook.php?keps1=0-10',None,None),
    ('pol_beg_2','polity','Beginner','NCERT','Political Theory','NCERT','Class 11','NCERT',None,None,'The why behind constitutional provisions — abstract concepts made concrete.','https://ncert.nic.in/textbook.php?keps2=0-9',None,None),
    ('pol_beg_3','polity','Beginner','NCERT','Contemporary World Politics','NCERT','Class 12','NCERT',None,None,'India in the global context — foreign policy and international relations.','https://ncert.nic.in/textbook.php?leps1=0-9',None,None),
    ('pol_beg_4','polity','Beginner','NCERT','Politics in India since Independence','NCERT','Class 12','NCERT',None,None,'India political history from 1947 to present.','https://ncert.nic.in/textbook.php?leps2=0-9',None,None),
    # Intermediate — Reference Books
    ('pol_int_1','polity','Intermediate','Book','Introduction to the Constitution of India','D.D. Basu','Reference Book','LexisNexis',2019,'978-8180382871','Standard law school reference for the Indian Constitution.','https://www.amazon.in/dp/8180382877',None,None),
    ('pol_int_2','polity','Intermediate','Book','Working a Democratic Constitution','Granville Austin','Reference Book','Oxford University Press',1999,'978-0195648539','Definitive political history of the Indian Constitution in practice.','https://www.amazon.in/dp/0195648536',None,None),
    ('pol_int_3','polity','Intermediate','Book','Indian Politics: A Reader','Zoya Hasan (Ed.)','Reference Book','Oxford University Press',2008,'978-0198061199','Leading political scientists on every dimension of Indian politics.','https://www.amazon.in/dp/0198061196',None,None),
    ('pol_int_4','polity','Intermediate','Book','India\'s Foreign Policy: Retrospect and Prospect','Srinath Raghavan','Reference Book','Oxford University Press',2010,'978-0198070702','Deepens NCERT contemporary world politics with historical depth.','https://www.amazon.in/dp/019807070X',None,None),
    # Advanced — Research Papers
    ('pol_adv_1','polity','Advanced','Paper','The Quasi-Federal Constitution? Taxonomical Influences on Interpretation of Federalism in India','Global Constitutionalism (Cambridge)','Research Paper',None,2025,None,'Challenges the quasi-federal label NCERT uses.','https://www.cambridge.org/core/journals/global-constitutionalism/article/quasifederal-constitution/FCD6BF65EB66D93C09B916B0E2E036A7','Global Constitutionalism, Cambridge University Press','Federalism — Class 11 Indian Constitution at Work Ch. 7'),
    ('pol_adv_2','polity','Advanced','Paper','Policing the Federation: The Supreme Court and Judicial Federalism in India','Swenden & Saxena','Research Paper',None,2022,None,'Qualitative study of 40 Supreme Court judgments on federalism.','https://www.research.ed.ac.uk/en/publications/policing-the-federation-the-supreme-court-and-judicial-federalism/','Territory, Politics, Governance, Taylor & Francis','Judiciary & federalism — Class 11 Ch. 6 & 7'),
    ('pol_adv_3','polity','Advanced','Paper','Federalism and Constitutional Issues in India — A Legal Analysis','IJNRD','Research Paper',None,2023,None,'Detailed legal analysis of landmark cases like Kesavananda Bharati and SR Bommai.','https://www.ijnrd.org/papers/IJNRDTH00196.pdf','International Journal of Novel Research and Development','Federalism — Class 11 Ch. 7'),

    # ── PSYCHOLOGY ───────────────────────────────────────────────────────
    # Beginner — NCERT
    ('psy_beg_1','psychology','Beginner','NCERT','Introduction to Psychology','NCERT','Class 11','NCERT',None,None,'Complete foundation — all major branches of psychology introduced.','https://ncert.nic.in/textbook.php?keps3=0-9',None,None),
    ('psy_beg_2','psychology','Beginner','NCERT','Psychology','NCERT','Class 12','NCERT',None,None,'Applied and clinical psychology — mental health, social behaviour, career skills.','https://ncert.nic.in/textbook.php?leps3=0-9',None,None),
    # Intermediate — Reference Books
    ('psy_int_1','psychology','Intermediate','Book','Psychology: Themes and Variations','Wayne Weiten','Reference Book','Cengage Learning',2016,'978-1305498205','Best undergraduate psychology textbook globally.','https://www.amazon.in/dp/1305498208',None,None),
    ('psy_int_2','psychology','Intermediate','Book','Abnormal Psychology: An Integrative Approach','David Barlow & V. Mark Durand','Reference Book','Cengage Learning',2018,'978-1305950443','Gold standard for psychological disorders — full clinical picture.','https://www.amazon.in/dp/1305950445',None,None),
    ('psy_int_3','psychology','Intermediate','Book','Influence: The Psychology of Persuasion','Robert Cialdini','Reference Book','Harper Business',1984,'978-0061241895','Makes NCERT social psychology chapter come alive.','https://www.amazon.in/dp/006124189X',None,None),
    ('psy_int_4','psychology','Intermediate','Book','Flourish','Martin Seligman','Reference Book','Free Press',2011,'978-1439190760','Extends NCERT chapter on meeting life challenges — positive psychology.','https://www.amazon.in/dp/1439190763',None,None),
    # Advanced — Research Papers
    ('psy_adv_1','psychology','Advanced','Paper','Cognitive, Social, and Physiological Determinants of Emotional State','Schachter & Singer','Research Paper',None,1962,None,'Classic experiment — emotions shaped by both physiology and interpretation.','https://psycnet.apa.org/doi/10.1037/h0046234','Psychological Review, APA','Motivation & emotion — Class 11 Ch. 9'),
    ('psy_adv_2','psychology','Advanced','Paper','Behavioral Study of Obedience','Stanley Milgram','Research Paper',None,1963,None,'Most famous social psychology experiment on obedience to authority.','https://psycnet.apa.org/doi/10.1037/h0040525','Journal of Abnormal and Social Psychology, APA','Social influence & group processes — Class 12 Ch. 6'),
    ('psy_adv_3','psychology','Advanced','Paper','A Theory of Human Motivation','Abraham Maslow','Research Paper',None,1943,None,'The original Maslow hierarchy of needs paper.','https://psycnet.apa.org/doi/10.1037/h0054346','Psychological Review, APA','Motivation & human needs — Class 11 Ch. 9'),
    ('psy_adv_4','psychology','Advanced','Paper','An Experimental Study of Apparent Behavior','Heider & Simmel','Research Paper',None,1944,None,'Foundational paper for attribution theory in social cognition.','https://doi.org/10.2307/1416950','American Journal of Psychology','Attitude & social cognition — Class 12 Ch. 5'),

    # ── BUSINESS STUDIES ─────────────────────────────────────────────────
    # Beginner — NCERT
    ('bs_beg_1','business_studies','Beginner','NCERT','Business Studies Part 1','NCERT','Class 11','NCERT',None,None,'Foundation — ownership structures, trade, services, and digital business.','https://ncert.nic.in/textbook.php?kebo1=0-9',None,None),
    ('bs_beg_2','business_studies','Beginner','NCERT','Business Studies Part 2','NCERT','Class 11','NCERT',None,None,'Company formation, finance, internal and international trade.','https://ncert.nic.in/textbook.php?kebo2=0-7',None,None),
    ('bs_beg_3','business_studies','Beginner','NCERT','Business Studies Part 1 — Principles & Functions of Management','NCERT','Class 12','NCERT',None,None,'Complete management theory — Fayol, Taylor, and all five functions.','https://ncert.nic.in/textbook.php?lebo1=0-8',None,None),
    ('bs_beg_4','business_studies','Beginner','NCERT','Business Studies Part 2 — Business Finance and Marketing','NCERT','Class 12','NCERT',None,None,'Capital decisions, stock market, marketing strategy, consumer rights.','https://ncert.nic.in/textbook.php?lebo2=0-6',None,None),
    # Intermediate — Reference Books
    ('bs_int_1','business_studies','Intermediate','Book','Principles of Management','P.C. Tripathi & P.N. Reddy','Reference Book','McGraw Hill',2019,'978-9353166403','Most popular Indian management textbook — extends NCERT with real examples.','https://www.amazon.in/dp/9353166403',None,None),
    ('bs_int_2','business_studies','Intermediate','Book','Marketing Management','Philip Kotler & Kevin Lane Keller','Reference Book','Pearson',2015,'978-9332557406','Definitive global marketing textbook — 4P to full strategic framework.','https://www.amazon.in/dp/9332557403',None,None),
    ('bs_int_3','business_studies','Intermediate','Book','Financial Management','I.M. Pandey','Reference Book','Vikas Publishing',2015,'978-8174465962','Standard Indian text — extends NCERT financial decisions chapter.','https://www.amazon.in/dp/8174465960',None,None),
    ('bs_int_4','business_studies','Intermediate','Book','Entrepreneurship: Successfully Launching New Ventures','Barringer & Ireland','Reference Book','Pearson',2015,'978-0134234809','Bridges NCERT entrepreneurship chapter to the real startup world.','https://www.amazon.in/dp/0134234804',None,None),
    # Advanced — Research Papers
    ('bs_adv_1','business_studies','Advanced','Paper','What is Strategy?','Michael E. Porter','Research Paper',None,1996,None,'Clarifies what strategy means — extends planning and business environment chapters.','https://hbr.org/1996/11/what-is-strategy','Harvard Business Review','Planning & business environment — Class 12 Ch. 3 & 4'),
    ('bs_adv_2','business_studies','Advanced','Paper','The Balanced Scorecard: Measures That Drive Performance','Kaplan & Norton','Research Paper',None,1992,None,'Revolutionised performance measurement — extends NCERT controlling chapter.','https://hbr.org/1992/01/the-balanced-scorecard-measures-that-drive-performance-2','Harvard Business Review','Controlling — Class 12 Ch. 8'),
    ('bs_adv_3','business_studies','Advanced','Paper','One More Time: How Do You Motivate Employees?','Frederick Herzberg','Research Paper',None,1968,None,'Herzberg two-factor theory — the original paper NCERT references.','https://hbr.org/2003/01/one-more-time-how-do-you-motivate-employees','Harvard Business Review','Directing & motivation — Class 12 Ch. 7'),
    ('bs_adv_4','business_studies','Advanced','Paper','The Market for Lemons: Quality Uncertainty and the Market Mechanism','George Akerlof','Research Paper',None,1970,None,'Nobel Prize-winning paper — economic rationale behind consumer protection.','https://doi.org/10.2307/1879431','Quarterly Journal of Economics, Oxford','Markets & consumer protection — Class 12 Ch. 11'),
]

cur.executemany("""
    INSERT OR IGNORE INTO recommendations
    (rec_id, subject_id, level, level_type, title, author, class, publisher,
     year, isbn, why, link, journal, ncert_chapter_link)
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)
""", recs)
conn.commit()
print(f'✅ {len(recs)} recommendations seeded')

## Step 6 — Seed Chapters (85 chapters across 5 subjects)

In [8]:
# FORMAT: (pdf_filename, zip_source, subject_id, class, chapter_number, chapter_name, book_name, is_special)
# zip_source matches the folder name inside ncert.zip — used to disambiguate
# Class 11 & 12 Business Studies have identical filenames, zip_source tells them apart
# is_special=1 for prelim (ps) and glossary (gl) files

chapters = [

    # ── CLASS 11 BUSINESS STUDIES (lebs1XX) ─────────────────────────────
    # 8 chapters + 1 prelim
    ('lebs101.pdf', 'class 11 business studies', 'business_studies', 11, 1,  'Business, Trade and Commerce',                            'Business Studies Part 1', 0),
    ('lebs102.pdf', 'class 11 business studies', 'business_studies', 11, 2,  'Forms of Business Organisations',                         'Business Studies Part 1', 0),
    ('lebs103.pdf', 'class 11 business studies', 'business_studies', 11, 3,  'Private, Public and Global Enterprises',                   'Business Studies Part 1', 0),
    ('lebs104.pdf', 'class 11 business studies', 'business_studies', 11, 4,  'Business Services',                                       'Business Studies Part 1', 0),
    ('lebs105.pdf', 'class 11 business studies', 'business_studies', 11, 5,  'Emerging Modes of Business',                              'Business Studies Part 2', 0),
    ('lebs106.pdf', 'class 11 business studies', 'business_studies', 11, 6,  'Social Responsibilities of Business and Business Ethics',  'Business Studies Part 2', 0),
    ('lebs107.pdf', 'class 11 business studies', 'business_studies', 11, 7,  'Formation of a Company',                                  'Business Studies Part 2', 0),
    ('lebs108.pdf', 'class 11 business studies', 'business_studies', 11, 8,  'Sources of Business Finance',                             'Business Studies Part 2', 0),
    ('lebs1ps.pdf', 'class 11 business studies', 'business_studies', 11, None, 'Prelim / Cover Pages',                                  'Business Studies',        1),

    # ── CLASS 11 ECONOMICS (keec1XX) ────────────────────────────────────
    # 8 chapters + glossary + prelim
    ('keec101.pdf', 'class 11 economics', 'economics', 11, 1,  'Indian Economy on the Eve of Independence',                   'Indian Economic Development', 0),
    ('keec102.pdf', 'class 11 economics', 'economics', 11, 2,  'Indian Economy 1950-1990',                                    'Indian Economic Development', 0),
    ('keec103.pdf', 'class 11 economics', 'economics', 11, 3,  'Liberalisation, Privatisation and Globalisation',             'Indian Economic Development', 0),
    ('keec104.pdf', 'class 11 economics', 'economics', 11, 4,  'Poverty',                                                     'Indian Economic Development', 0),
    ('keec105.pdf', 'class 11 economics', 'economics', 11, 5,  'Human Capital Formation in India',                            'Indian Economic Development', 0),
    ('keec106.pdf', 'class 11 economics', 'economics', 11, 6,  'Rural Development',                                           'Indian Economic Development', 0),
    ('keec107.pdf', 'class 11 economics', 'economics', 11, 7,  'Employment: Growth, Informalisation and Other Issues',        'Indian Economic Development', 0),
    ('keec108.pdf', 'class 11 economics', 'economics', 11, 8,  'Infrastructure',                                              'Indian Economic Development', 0),
    ('keec1gl.pdf', 'class 11 economics', 'economics', 11, None, 'Glossary',                                                  'Indian Economic Development', 1),
    ('keec1ps.pdf', 'class 11 economics', 'economics', 11, None, 'Prelim / Cover Pages',                                      'Indian Economic Development', 1),

    # ── CLASS 11 POLITY (keps1XX) ────────────────────────────────────────
    # 8 chapters + prelim
    ('keps101.pdf', 'class 11 polity', 'polity', 11, 1,  'Constitution: Why and How?',                                  'Indian Constitution at Work', 0),
    ('keps102.pdf', 'class 11 polity', 'polity', 11, 2,  'Rights in the Indian Constitution',                           'Indian Constitution at Work', 0),
    ('keps103.pdf', 'class 11 polity', 'polity', 11, 3,  'Election and Representation',                                 'Indian Constitution at Work', 0),
    ('keps104.pdf', 'class 11 polity', 'polity', 11, 4,  'Executive',                                                   'Indian Constitution at Work', 0),
    ('keps105.pdf', 'class 11 polity', 'polity', 11, 5,  'Legislature',                                                 'Indian Constitution at Work', 0),
    ('keps106.pdf', 'class 11 polity', 'polity', 11, 6,  'Judiciary',                                                   'Indian Constitution at Work', 0),
    ('keps107.pdf', 'class 11 polity', 'polity', 11, 7,  'Federalism',                                                  'Indian Constitution at Work', 0),
    ('keps108.pdf', 'class 11 polity', 'polity', 11, 8,  'Local Governments',                                           'Indian Constitution at Work', 0),
    ('keps1ps.pdf', 'class 11 polity', 'polity', 11, None, 'Prelim / Cover Pages',                                      'Indian Constitution at Work', 1),

    # ── CLASS 11 PSYCHOLOGY (kepy1XX) ────────────────────────────────────
    # 8 chapters + glossary + prelim
    ('kepy101.pdf', 'class 11 psychology', 'psychology', 11, 1,  'What is Psychology?',                                   'Introduction to Psychology', 0),
    ('kepy102.pdf', 'class 11 psychology', 'psychology', 11, 2,  'Methods of Enquiry in Psychology',                      'Introduction to Psychology', 0),
    ('kepy103.pdf', 'class 11 psychology', 'psychology', 11, 3,  'The Bases of Human Behaviour',                          'Introduction to Psychology', 0),
    ('kepy104.pdf', 'class 11 psychology', 'psychology', 11, 4,  'Human Development',                                     'Introduction to Psychology', 0),
    ('kepy105.pdf', 'class 11 psychology', 'psychology', 11, 5,  'Sensory, Attentional and Perceptual Processes',          'Introduction to Psychology', 0),
    ('kepy106.pdf', 'class 11 psychology', 'psychology', 11, 6,  'Learning',                                              'Introduction to Psychology', 0),
    ('kepy107.pdf', 'class 11 psychology', 'psychology', 11, 7,  'Human Memory',                                          'Introduction to Psychology', 0),
    ('kepy108.pdf', 'class 11 psychology', 'psychology', 11, 8,  'Thinking',                                              'Introduction to Psychology', 0),
    ('kepy1gl.pdf', 'class 11 psychology', 'psychology', 11, None, 'Glossary',                                            'Introduction to Psychology', 1),
    ('kepy1ps.pdf', 'class 11 psychology', 'psychology', 11, None, 'Prelim / Cover Pages',                                'Introduction to Psychology', 1),

    # ── CLASS 11 SOCIOLOGY (kesy1XX) ─────────────────────────────────────
    # 5 chapters + prelim
    ('kesy101.pdf', 'class 11 sociology', 'sociology', 11, 1,  'Society and Sociology',                                   'Introducing Sociology', 0),
    ('kesy102.pdf', 'class 11 sociology', 'sociology', 11, 2,  'Terms, Concepts and their Use in Sociology',              'Introducing Sociology', 0),
    ('kesy103.pdf', 'class 11 sociology', 'sociology', 11, 3,  'Understanding Social Institutions',                       'Introducing Sociology', 0),
    ('kesy104.pdf', 'class 11 sociology', 'sociology', 11, 4,  'Culture and Socialisation',                               'Introducing Sociology', 0),
    ('kesy105.pdf', 'class 11 sociology', 'sociology', 11, 5,  'Doing Sociology: Research Methods',                       'Introducing Sociology', 0),
    ('kesy1ps.pdf', 'class 11 sociology', 'sociology', 11, None, 'Prelim / Cover Pages',                                  'Introducing Sociology', 1),

    # ── CLASS 12 BUSINESS STUDIES (lebs1XX — same filenames as class 11!) ─
    # zip_source distinguishes these from class 11 entries above
    ('lebs101.pdf', 'class 12 business studies', 'business_studies', 12, 1,  'Nature and Significance of Management',      'Business Studies Part 1', 0),
    ('lebs102.pdf', 'class 12 business studies', 'business_studies', 12, 2,  'Principles of Management',                   'Business Studies Part 1', 0),
    ('lebs103.pdf', 'class 12 business studies', 'business_studies', 12, 3,  'Business Environment',                       'Business Studies Part 1', 0),
    ('lebs104.pdf', 'class 12 business studies', 'business_studies', 12, 4,  'Planning',                                   'Business Studies Part 1', 0),
    ('lebs105.pdf', 'class 12 business studies', 'business_studies', 12, 5,  'Organising',                                 'Business Studies Part 1', 0),
    ('lebs106.pdf', 'class 12 business studies', 'business_studies', 12, 6,  'Staffing',                                   'Business Studies Part 1', 0),
    ('lebs107.pdf', 'class 12 business studies', 'business_studies', 12, 7,  'Directing',                                  'Business Studies Part 1', 0),
    ('lebs108.pdf', 'class 12 business studies', 'business_studies', 12, 8,  'Controlling',                                'Business Studies Part 1', 0),
    ('lebs1ps.pdf', 'class 12 business studies', 'business_studies', 12, None, 'Prelim / Cover Pages',                    'Business Studies',         1),

    # ── CLASS 12 ECONOMICS (leec1XX) ─────────────────────────────────────
    # 6 chapters + prelim
    ('leec101.pdf', 'class 12 economics', 'economics', 12, 1,  'Introduction to Macroeconomics',                          'Introductory Macroeconomics', 0),
    ('leec102.pdf', 'class 12 economics', 'economics', 12, 2,  'National Income Accounting',                              'Introductory Macroeconomics', 0),
    ('leec103.pdf', 'class 12 economics', 'economics', 12, 3,  'Money and Banking',                                       'Introductory Macroeconomics', 0),
    ('leec104.pdf', 'class 12 economics', 'economics', 12, 4,  'Income Determination',                                    'Introductory Macroeconomics', 0),
    ('leec105.pdf', 'class 12 economics', 'economics', 12, 5,  'The Government: Budget and the Economy',                  'Introductory Macroeconomics', 0),
    ('leec106.pdf', 'class 12 economics', 'economics', 12, 6,  'Open Economy Macroeconomics',                             'Introductory Macroeconomics', 0),
    ('leec1ps.pdf', 'class 12 economics', 'economics', 12, None, 'Prelim / Cover Pages',                                  'Introductory Macroeconomics', 1),

    # ── CLASS 12 POLITY (leps1XX) ────────────────────────────────────────
    # 7 chapters + prelim
    ('leps101.pdf', 'class 12 polity', 'polity', 12, 1,  'Challenges of Nation Building',                               'Politics in India since Independence', 0),
    ('leps102.pdf', 'class 12 polity', 'polity', 12, 2,  'Era of One-Party Dominance',                                  'Politics in India since Independence', 0),
    ('leps103.pdf', 'class 12 polity', 'polity', 12, 3,  'Politics of Planned Development',                             'Politics in India since Independence', 0),
    ('leps104.pdf', 'class 12 polity', 'polity', 12, 4,  "India's External Relations",                                  'Politics in India since Independence', 0),
    ('leps105.pdf', 'class 12 polity', 'polity', 12, 5,  'Challenges to and Restoration of the Congress System',        'Politics in India since Independence', 0),
    ('leps106.pdf', 'class 12 polity', 'polity', 12, 6,  'Crisis of the Constitutional Order',                          'Politics in India since Independence', 0),
    ('leps107.pdf', 'class 12 polity', 'polity', 12, 7,  'Rise of Popular Movements',                                   'Politics in India since Independence', 0),
    ('leps1ps.pdf', 'class 12 polity', 'polity', 12, None, 'Prelim / Cover Pages',                                      'Politics in India since Independence', 1),

    # ── CLASS 12 PSYCHOLOGY (lepy1XX) ────────────────────────────────────
    # 7 chapters + glossary + prelim
    ('lepy101.pdf', 'class 12 psychology', 'psychology', 12, 1,  'Intelligent Behaviour',                                 'Psychology', 0),
    ('lepy102.pdf', 'class 12 psychology', 'psychology', 12, 2,  'Self and Personality',                                  'Psychology', 0),
    ('lepy103.pdf', 'class 12 psychology', 'psychology', 12, 3,  'Meeting Life Challenges',                               'Psychology', 0),
    ('lepy104.pdf', 'class 12 psychology', 'psychology', 12, 4,  'Psychological Disorders',                               'Psychology', 0),
    ('lepy105.pdf', 'class 12 psychology', 'psychology', 12, 5,  'Therapeutic Approaches',                                'Psychology', 0),
    ('lepy106.pdf', 'class 12 psychology', 'psychology', 12, 6,  'Attitude and Social Cognition',                         'Psychology', 0),
    ('lepy107.pdf', 'class 12 psychology', 'psychology', 12, 7,  'Social Influence and Group Processes',                  'Psychology', 0),
    ('lepy1gl.pdf', 'class 12 psychology', 'psychology', 12, None, 'Glossary',                                            'Psychology', 1),
    ('lepy1ps.pdf', 'class 12 psychology', 'psychology', 12, None, 'Prelim / Cover Pages',                                'Psychology', 1),

    # ── CLASS 12 SOCIOLOGY (kegy2XX) ─────────────────────────────────────
    # NOTE: uses kegy2 prefix (not lesy!) — 14 chapters + glossary + prelim
    ('kegy201.pdf', 'class 12 sociology', 'sociology', 12, 1,  'Introducing Indian Society',                              'Indian Society', 0),
    ('kegy202.pdf', 'class 12 sociology', 'sociology', 12, 2,  'The Demographic Structure of the Indian Society',         'Indian Society', 0),
    ('kegy203.pdf', 'class 12 sociology', 'sociology', 12, 3,  'Social Institutions: Continuity and Change',             'Indian Society', 0),
    ('kegy204.pdf', 'class 12 sociology', 'sociology', 12, 4,  'Market as a Social Institution',                         'Indian Society', 0),
    ('kegy205.pdf', 'class 12 sociology', 'sociology', 12, 5,  'Patterns of Social Inequality and Exclusion',            'Indian Society', 0),
    ('kegy206.pdf', 'class 12 sociology', 'sociology', 12, 6,  'The Challenges of Cultural Diversity',                   'Indian Society', 0),
    ('kegy207.pdf', 'class 12 sociology', 'sociology', 12, 7,  'Structural Change',                                      'Social Change and Development in India', 0),
    ('kegy208.pdf', 'class 12 sociology', 'sociology', 12, 8,  'Cultural Change',                                        'Social Change and Development in India', 0),
    ('kegy209.pdf', 'class 12 sociology', 'sociology', 12, 9,  'The Story of Indian Democracy',                          'Social Change and Development in India', 0),
    ('kegy210.pdf', 'class 12 sociology', 'sociology', 12, 10, 'Change and Development in Rural Society',                'Social Change and Development in India', 0),
    ('kegy211.pdf', 'class 12 sociology', 'sociology', 12, 11, 'Change and Development in Industrial Society',           'Social Change and Development in India', 0),
    ('kegy212.pdf', 'class 12 sociology', 'sociology', 12, 12, 'Globalisation and Social Change',                        'Social Change and Development in India', 0),
    ('kegy213.pdf', 'class 12 sociology', 'sociology', 12, 13, 'Mass Media and Communications',                          'Social Change and Development in India', 0),
    ('kegy214.pdf', 'class 12 sociology', 'sociology', 12, 14, 'Social Movements',                                       'Social Change and Development in India', 0),
    ('kegy2gl.pdf', 'class 12 sociology', 'sociology', 12, None, 'Glossary',                                             'Social Change and Development in India', 1),
    ('kegy2ps.pdf', 'class 12 sociology', 'sociology', 12, None, 'Prelim / Cover Pages',                                 'Social Change and Development in India', 1),
]

cur.executemany("""
    INSERT OR IGNORE INTO chapters
    (pdf_filename, zip_source, subject_id, class, chapter_number, chapter_name, book_name, is_special)
    VALUES (?,?,?,?,?,?,?,?)
""", chapters)
conn.commit()
print(f'✅ {len(chapters)} chapter entries seeded')
print(f'   ({len([c for c in chapters if not c[7]])} real chapters + {len([c for c in chapters if c[7]])} prelim/glossary files)')

✅ 93 chapter entries seeded
   (79 real chapters + 14 prelim/glossary files)


In [9]:
"""
NCERT Topics — Manual Insert Script
Covers all 5 subjects × Class 11 & 12
Run this in your notebook with an active `conn` (sqlite3 connection)
"""

topics = [

    # ══════════════════════════════════════════════════════════════
    # ECONOMICS — Class 11: Indian Economic Development
    # ══════════════════════════════════════════════════════════════
    # Ch 1
    ("Indian Economy on the Eve of Independence",                       "keec101.pdf"),
    ("Colonial economic policies and their impact",                     "keec101.pdf"),
    ("De-industrialisation under British rule",                        "keec101.pdf"),
    ("Stagnation in agriculture under colonialism",                    "keec101.pdf"),
    ("Demographic profile at Independence",                            "keec101.pdf"),
    ("Infrastructure left by the British",                             "keec101.pdf"),

    # Ch 2
    ("Indian Economy 1950–1990",                                       "keec102.pdf"),
    ("Five Year Plans — goals and strategy",                           "keec102.pdf"),
    ("Role of the public sector in early India",                       "keec102.pdf"),
    ("Green Revolution — causes and consequences",                     "keec102.pdf"),
    ("Industrial policy and licensing regime",                         "keec102.pdf"),
    ("Trade policy — import substitution",                             "keec102.pdf"),

    # Ch 3
    ("Liberalisation, Privatisation and Globalisation (LPG reforms)",  "keec103.pdf"),
    ("Balance of payments crisis 1991",                                "keec103.pdf"),
    ("Structural adjustment and IMF conditionalities",                 "keec103.pdf"),
    ("Disinvestment and privatisation debate",                         "keec103.pdf"),
    ("Globalisation — benefits and criticisms",                        "keec103.pdf"),
    ("WTO and India's trade commitments",                              "keec103.pdf"),

    # Ch 4
    ("Poverty — absolute and relative concepts",                       "keec104.pdf"),
    ("Poverty line — calorie-based measurement",                       "keec104.pdf"),
    ("Poverty estimation in India — Tendulkar Committee",              "keec104.pdf"),
    ("Causes of poverty in India",                                     "keec104.pdf"),
    ("Anti-poverty programmes — MGNREGA, PDS",                        "keec104.pdf"),
    ("Poverty trends since 1991",                                      "keec104.pdf"),

    # Ch 5
    ("Human Capital Formation in India",                               "keec105.pdf"),
    ("Education as investment vs consumption",                         "keec105.pdf"),
    ("Role of health in human capital",                                "keec105.pdf"),
    ("Government expenditure on education and health",                 "keec105.pdf"),
    ("Brain drain — causes and effects",                               "keec105.pdf"),

    # Ch 6
    ("Rural Development — meaning and objectives",                     "keec106.pdf"),
    ("Agricultural credit — formal and informal sources",              "keec106.pdf"),
    ("Land reforms in India",                                          "keec106.pdf"),
    ("Agricultural marketing — MSP and APMCs",                        "keec106.pdf"),
    ("Diversification into animal husbandry and horticulture",         "keec106.pdf"),
    ("Organic farming",                                                "keec106.pdf"),

    # Ch 7
    ("Employment — formal and informal sector",                        "keec107.pdf"),
    ("Worker population ratio and labour force participation",         "keec107.pdf"),
    ("Informalisation of workforce",                                   "keec107.pdf"),
    ("Unemployment — types and measurement",                           "keec107.pdf"),
    ("Women and work in India",                                        "keec107.pdf"),

    # Ch 8
    ("Infrastructure — economic and social",                           "keec108.pdf"),
    ("Energy sector — conventional and non-conventional",              "keec108.pdf"),
    ("Health infrastructure in India",                                 "keec108.pdf"),
    ("India vs China — infrastructure comparison",                     "keec108.pdf"),

    # ══════════════════════════════════════════════════════════════
    # ECONOMICS — Class 12: Introductory Macroeconomics
    # ══════════════════════════════════════════════════════════════
    # Ch 1
    ("Introduction to Macroeconomics — circular flow",                 "leec101.pdf"),
    ("Stocks and flows",                                               "leec101.pdf"),
    ("Micro vs Macro economics",                                       "leec101.pdf"),

    # Ch 2
    ("National Income Accounting — GDP, GNP, NNP",                    "leec102.pdf"),
    ("Value added method of calculating national income",              "leec102.pdf"),
    ("Expenditure method — C + I + G + NX",                           "leec102.pdf"),
    ("Income method of national income",                               "leec102.pdf"),
    ("Real vs Nominal GDP — GDP deflator",                             "leec102.pdf"),
    ("Green GDP and limitations of GDP",                               "leec102.pdf"),

    # Ch 3
    ("Money — functions and types",                                    "leec103.pdf"),
    ("Money supply — M1, M2, M3, M4",                                 "leec103.pdf"),
    ("Credit creation by commercial banks",                            "leec103.pdf"),
    ("Central bank — functions of RBI",                                "leec103.pdf"),
    ("Monetary policy — tools and objectives",                         "leec103.pdf"),

    # Ch 4
    ("Aggregate demand and aggregate supply",                          "leec104.pdf"),
    ("Consumption function and Marginal Propensity to Consume",        "leec104.pdf"),
    ("Investment function",                                            "leec104.pdf"),
    ("Keynesian multiplier",                                           "leec104.pdf"),
    ("Inflationary and deflationary gap",                              "leec104.pdf"),
    ("Paradox of thrift",                                              "leec104.pdf"),

    # Ch 5
    ("Government budget — components and types",                       "leec105.pdf"),
    ("Revenue vs capital receipts and expenditure",                    "leec105.pdf"),
    ("Fiscal deficit, revenue deficit, primary deficit",               "leec105.pdf"),
    ("Fiscal policy — expansionary and contractionary",                "leec105.pdf"),
    ("FRBM Act and fiscal consolidation",                              "leec105.pdf"),

    # Ch 6
    ("Balance of Payments — current and capital account",              "leec106.pdf"),
    ("Foreign exchange rate — fixed vs flexible",                      "leec106.pdf"),
    ("Purchasing Power Parity",                                        "leec106.pdf"),
    ("Managed float exchange rate system",                             "leec106.pdf"),
    ("Foreign exchange market — demand and supply",                    "leec106.pdf"),

    # ══════════════════════════════════════════════════════════════
    # POLITY — Class 11: Indian Constitution at Work
    # ══════════════════════════════════════════════════════════════
    ("Constitution — meaning, need and making",                        "keps101.pdf"),
    ("Constituent Assembly debates",                                   "keps101.pdf"),
    ("Preamble — ideals and philosophy",                               "keps101.pdf"),
    ("Fundamental Rights — Articles 12–35",                           "keps102.pdf"),
    ("Right to Equality — Articles 14–18",                            "keps102.pdf"),
    ("Right to Freedom — Article 19–22",                              "keps102.pdf"),
    ("Right to Constitutional Remedies — Article 32",                 "keps102.pdf"),
    ("Directive Principles of State Policy",                           "keps102.pdf"),
    ("Fundamental Duties",                                             "keps102.pdf"),
    ("Election Commission and electoral process",                      "keps103.pdf"),
    ("First Past the Post vs Proportional Representation",             "keps103.pdf"),
    ("Anti-defection law",                                             "keps103.pdf"),
    ("Parliament — Lok Sabha and Rajya Sabha",                        "keps104.pdf"),
    ("President — powers, election and role",                          "keps104.pdf"),
    ("Prime Minister and Council of Ministers",                        "keps104.pdf"),
    ("Parliament — law-making process",                                "keps105.pdf"),
    ("Parliamentary committees",                                       "keps105.pdf"),
    ("Budget presentation in Parliament",                              "keps105.pdf"),
    ("Supreme Court — jurisdiction and judicial review",               "keps106.pdf"),
    ("High Courts and subordinate courts",                             "keps106.pdf"),
    ("Independence of judiciary",                                      "keps106.pdf"),
    ("PIL — Public Interest Litigation",                               "keps106.pdf"),
    ("Federalism — features and types",                                "keps107.pdf"),
    ("Union and State lists — division of powers",                     "keps107.pdf"),
    ("Centre-State relations",                                         "keps107.pdf"),
    ("Emergency provisions — Articles 352, 356, 360",                 "keps107.pdf"),
    ("Panchayati Raj — 73rd and 74th Amendments",                     "keps108.pdf"),
    ("Three-tier local self-government",                               "keps108.pdf"),
    ("Reservation in local bodies",                                    "keps108.pdf"),

    # ══════════════════════════════════════════════════════════════
    # POLITY — Class 12
    # ══════════════════════════════════════════════════════════════
    # Politics in India since Independence
    ("Challenges of Nation Building — partition and integration",      "leps101.pdf"),
    ("Integration of princely states",                                 "leps101.pdf"),
    ("Reorganisation of states on linguistic basis",                   "leps101.pdf"),
    ("Era of One-Party Dominance — Congress system",                   "leps102.pdf"),
    ("Nature of Congress dominance",                                   "leps102.pdf"),
    ("Opposition parties in early India",                              "leps102.pdf"),
    ("Politics of Planned Development",                                "leps103.pdf"),
    ("Kerala experiment — communist government 1957",                  "leps103.pdf"),
    ("India's External Relations — Non-Alignment Movement",           "leps104.pdf"),
    ("India-China war 1962",                                           "leps104.pdf"),
    ("India-Pakistan wars 1965 and 1971",                              "leps104.pdf"),
    ("Challenges to Congress system — split and revival",              "leps105.pdf"),
    ("Green Revolution and political change",                          "leps105.pdf"),
    ("National Emergency 1975 — causes and consequences",              "leps106.pdf"),
    ("Restoration of democracy — Janata government",                   "leps106.pdf"),
    ("Rise of Popular Movements — anti-arrack, Chipko",               "leps107.pdf"),
    ("Dalit movements and backward class politics",                    "leps107.pdf"),

    # Contemporary World Politics
    ("Cold War era — bipolarity and its logic",                        "leps101.pdf"),
    ("Collapse of Soviet Union",                                       "leps101.pdf"),
    ("US hegemony in world politics",                                  "leps101.pdf"),
    ("Alternative centres of power — EU, ASEAN, China",               "leps101.pdf"),
    ("South Asia and international politics",                          "leps101.pdf"),
    ("United Nations — reforms and role",                              "leps101.pdf"),
    ("Security — traditional and non-traditional",                     "leps101.pdf"),
    ("Environment in world politics — Rio, Kyoto",                    "leps101.pdf"),
    ("Globalisation and its critics",                                  "leps101.pdf"),
    ("New world order — India's foreign policy",                       "leps101.pdf"),

    # ══════════════════════════════════════════════════════════════
    # PSYCHOLOGY — Class 11: Introduction to Psychology
    # ══════════════════════════════════════════════════════════════
    ("What is Psychology? — definition and branches",                  "kepy101.pdf"),
    ("Psychology as a natural and social science",                     "kepy101.pdf"),
    ("Methods of Enquiry — observation, experiment, survey",          "kepy102.pdf"),
    ("Case study and psychological testing",                           "kepy102.pdf"),
    ("Ethical issues in psychological research",                       "kepy102.pdf"),
    ("Nervous system — neurons and transmission",                      "kepy103.pdf"),
    ("Endocrine system and behaviour",                                 "kepy103.pdf"),
    ("Genetic basis of behaviour",                                     "kepy103.pdf"),
    ("Human Development — stages across lifespan",                    "kepy104.pdf"),
    ("Piaget's cognitive development theory",                          "kepy104.pdf"),
    ("Vygotsky's sociocultural theory",                                "kepy104.pdf"),
    ("Adolescence — identity and challenges",                          "kepy104.pdf"),
    ("Sensation — threshold and signal detection",                     "kepy105.pdf"),
    ("Attention — selective, divided and sustained",                   "kepy105.pdf"),
    ("Perception — principles and illusions",                          "kepy105.pdf"),
    ("Learning — classical conditioning (Pavlov)",                     "kepy106.pdf"),
    ("Operant conditioning — Skinner",                                 "kepy106.pdf"),
    ("Observational learning — Bandura",                               "kepy106.pdf"),
    ("Cognitive learning and insight",                                 "kepy106.pdf"),
    ("Human Memory — encoding, storage, retrieval",                   "kepy107.pdf"),
    ("Short-term and long-term memory",                                "kepy107.pdf"),
    ("Forgetting — theories and causes",                               "kepy107.pdf"),
    ("Thinking — concepts and reasoning",                              "kepy108.pdf"),
    ("Problem solving and creativity",                                 "kepy108.pdf"),
    ("Language and thought",                                           "kepy108.pdf"),

    # ══════════════════════════════════════════════════════════════
    # PSYCHOLOGY — Class 12: Psychology
    # ══════════════════════════════════════════════════════════════
    ("Intelligent Behaviour — theories of intelligence",               "lepy101.pdf"),
    ("Gardner's multiple intelligences",                               "lepy101.pdf"),
    ("IQ measurement and cultural bias",                               "lepy101.pdf"),
    ("Emotional intelligence",                                         "lepy101.pdf"),
    ("Self-concept and Self-esteem",                                   "lepy102.pdf"),
    ("Personality — type and trait approaches",                        "lepy102.pdf"),
    ("Freudian psychoanalytic theory",                                 "lepy102.pdf"),
    ("Humanistic approach — Rogers and Maslow",                        "lepy102.pdf"),
    ("Stress — sources and types",                                     "lepy103.pdf"),
    ("Coping with stress — strategies",                                "lepy103.pdf"),
    ("Health psychology — lifestyle and illness",                      "lepy103.pdf"),
    ("Psychological Disorders — classification (DSM/ICD)",            "lepy104.pdf"),
    ("Anxiety disorders",                                              "lepy104.pdf"),
    ("Mood disorders — depression and bipolar",                        "lepy104.pdf"),
    ("Schizophrenia and psychosis",                                    "lepy104.pdf"),
    ("Psychotherapy — psychoanalytic, humanistic",                     "lepy105.pdf"),
    ("Behaviour therapy and CBT",                                      "lepy105.pdf"),
    ("Biological therapies",                                           "lepy105.pdf"),
    ("Attitude — formation and change",                                "lepy106.pdf"),
    ("Prejudice and discrimination",                                   "lepy106.pdf"),
    ("Attribution — internal vs external",                             "lepy106.pdf"),
    ("Social Influence — conformity and obedience",                    "lepy107.pdf"),
    ("Group dynamics and leadership",                                  "lepy107.pdf"),
    ("Intergroup conflict and cooperation",                            "lepy107.pdf"),

    # ══════════════════════════════════════════════════════════════
    # SOCIOLOGY — Class 11: Introducing Sociology
    # ══════════════════════════════════════════════════════════════
    ("Society and Sociology — emergence of sociology",                 "kesy101.pdf"),
    ("Social facts — Durkheim",                                        "kesy101.pdf"),
    ("Sociological imagination — C. Wright Mills",                    "kesy101.pdf"),
    ("Terms, Concepts — status, role, norms, values",                 "kesy102.pdf"),
    ("Culture — components and ethnocentrism",                        "kesy102.pdf"),
    ("Social stratification — class, caste, gender",                  "kesy102.pdf"),
    ("Social Institutions — family, marriage, kinship",               "kesy103.pdf"),
    ("Types of family — nuclear, joint, extended",                    "kesy103.pdf"),
    ("Religion as social institution",                                 "kesy103.pdf"),
    ("Education as social institution",                                "kesy103.pdf"),
    ("Culture and Socialisation — primary and secondary",             "kesy104.pdf"),
    ("Agents of socialisation — family, peer, media",                 "kesy104.pdf"),
    ("Resocialisation and total institutions",                         "kesy104.pdf"),
    ("Research Methods — qualitative and quantitative",               "kesy105.pdf"),
    ("Survey, ethnography, and interview methods",                     "kesy105.pdf"),
    ("Ethical considerations in sociological research",               "kesy105.pdf"),

    # ══════════════════════════════════════════════════════════════
    # SOCIOLOGY — Class 12: Indian Society
    # ══════════════════════════════════════════════════════════════
    ("Introducing Indian Society — diversity and unity",               "kegy201.pdf"),
    ("Colonial impact on Indian society",                              "kegy201.pdf"),
    ("Demographic Structure of Indian Society",                        "kegy202.pdf"),
    ("Census and population data in India",                            "kegy202.pdf"),
    ("Demographic dividend",                                           "kegy202.pdf"),
    ("Social Institutions — caste system",                             "kegy203.pdf"),
    ("Tribe as a social category",                                     "kegy203.pdf"),
    ("Family and kinship in India",                                    "kegy203.pdf"),
    ("Market as a Social Institution",                                 "kegy204.pdf"),
    ("Capitalism and market society",                                  "kegy204.pdf"),
    ("Patterns of Social Inequality — caste, class, gender",          "kegy205.pdf"),
    ("Dalit identity and social exclusion",                            "kegy205.pdf"),
    ("Gender inequality in India",                                     "kegy205.pdf"),
    ("Disability and marginalisation",                                 "kegy205.pdf"),
    ("The Challenges of Cultural Diversity",                           "kegy206.pdf"),
    ("Communalism and religious violence",                             "kegy206.pdf"),
    ("Regionalism and ethnic movements",                               "kegy206.pdf"),

    # ══════════════════════════════════════════════════════════════
    # SOCIOLOGY — Class 12: Social Change and Development in India
    # ══════════════════════════════════════════════════════════════
    ("Structural Change — colonialism and urbanisation",               "kegy207.pdf"),
    ("Industrialisation and social transformation",                    "kegy207.pdf"),
    ("Cultural Change — Sanskritisation and Westernisation",          "kegy208.pdf"),
    ("Modernisation and social change",                                "kegy208.pdf"),
    ("The Story of Indian Democracy",                                  "kegy209.pdf"),
    ("Political parties and electoral politics",                       "kegy209.pdf"),
    ("Change and Development in Rural Society",                        "kegy210.pdf"),
    ("Land reforms and green revolution effects",                      "kegy210.pdf"),
    ("Caste in rural India",                                           "kegy210.pdf"),
    ("Change and Development in Industrial Society",                   "kegy211.pdf"),
    ("Trade unions and labour movements",                              "kegy211.pdf"),
    ("Globalisation and Industrial Society",                           "kegy212.pdf"),
    ("Mass Media and Communications",                                  "kegy213.pdf"),
    ("Social Media and its sociological impact",                       "kegy213.pdf"),
    ("Social Movements — old and new",                                 "kegy214.pdf"),
    ("Environmental movements in India",                               "kegy214.pdf"),
    ("Women's movements in India",                                     "kegy214.pdf"),

    # ══════════════════════════════════════════════════════════════
    # BUSINESS STUDIES — Class 11
    # ══════════════════════════════════════════════════════════════
    ("Business, Trade and Commerce — meaning and objectives",          "lebs101.pdf"),
    ("Characteristics of business activity",                           "lebs101.pdf"),
    ("Forms of Business Organisations — sole proprietorship",         "lebs102.pdf"),
    ("Partnership — features and types",                               "lebs102.pdf"),
    ("Joint Hindu Family business",                                    "lebs102.pdf"),
    ("Cooperative societies",                                          "lebs102.pdf"),
    ("Joint stock company — public and private",                       "lebs102.pdf"),
    ("Private, Public and Global Enterprises",                         "lebs103.pdf"),
    ("Public sector enterprises — forms and role",                     "lebs103.pdf"),
    ("Multinational corporations",                                     "lebs103.pdf"),
    ("Business Services — banking, insurance, transport",             "lebs104.pdf"),
    ("Warehousing and communication services",                         "lebs104.pdf"),
    ("Emerging Modes of Business — e-commerce",                       "lebs105.pdf"),
    ("B2B, B2C, C2C models",                                          "lebs105.pdf"),
    ("Outsourcing and its types",                                      "lebs105.pdf"),
    ("Social Responsibilities of Business",                            "lebs106.pdf"),
    ("Business Ethics",                                                "lebs106.pdf"),
    ("Formation of a Company — steps and documents",                   "lebs107.pdf"),
    ("Memorandum and Articles of Association",                         "lebs107.pdf"),
    ("Sources of Business Finance — equity, debt",                     "lebs108.pdf"),
    ("Internal and external sources of finance",                       "lebs108.pdf"),
    ("GDR, ADR and commercial paper",                                  "lebs108.pdf"),

    # ══════════════════════════════════════════════════════════════
    # BUSINESS STUDIES — Class 12
    # ══════════════════════════════════════════════════════════════
    ("Nature and Significance of Management",                          "lebs101.pdf"),
    ("Management as science, art and profession",                      "lebs101.pdf"),
    ("Levels of management",                                           "lebs101.pdf"),
    ("Coordination — essence of management",                           "lebs101.pdf"),
    ("Principles of Management — Fayol's 14 principles",              "lebs102.pdf"),
    ("Taylor's Scientific Management",                                 "lebs102.pdf"),
    ("Business Environment — components and importance",               "lebs103.pdf"),
    ("PESTLE analysis",                                                "lebs103.pdf"),
    ("Demonetisation and GST as business environment change",         "lebs103.pdf"),
    ("Planning — importance, types and limitations",                   "lebs104.pdf"),
    ("MBO — Management by Objectives",                                 "lebs104.pdf"),
    ("Organising — span of control and delegation",                    "lebs105.pdf"),
    ("Formal vs informal organisation",                                "lebs105.pdf"),
    ("Centralisation vs decentralisation",                             "lebs105.pdf"),
    ("Staffing — recruitment and selection",                           "lebs106.pdf"),
    ("Training and development methods",                               "lebs106.pdf"),
    ("Directing — leadership styles",                                  "lebs107.pdf"),
    ("Maslow's hierarchy of needs",                                    "lebs107.pdf"),
    ("Herzberg's two-factor theory",                                   "lebs107.pdf"),
    ("Communication — formal and informal channels",                   "lebs107.pdf"),
    ("Controlling — process and techniques",                           "lebs108.pdf"),
    ("PERT, CPM and ROI as control tools",                             "lebs108.pdf"),
]

# ── INSERT INTO DB ─────────────────────────────────────────────────────────
print(f"Total topics to insert: {len(topics)}\n")

inserted = 0
skipped  = 0

for topic_text, source_pdf in topics:
    try:
        conn.execute(
            "INSERT OR IGNORE INTO topics (topic_text, source_pdf) VALUES (?, ?)",
            (topic_text, source_pdf)
        )
        changed = conn.execute("SELECT changes()").fetchone()[0]
        if changed:
            inserted += 1
        else:
            skipped += 1
    except Exception as e:
        print(f"  ⚠️  Error on '{topic_text}': {e}")

conn.commit()
print(f"✅ Inserted : {inserted}")
print(f"⏭️  Skipped  : {skipped} (already existed)")

# ── VERIFY — count by subject ──────────────────────────────────────────────
print("\n── TOPICS BY SUBJECT ─────────────────────────────────")
result = conn.execute("""
    SELECT c.subject_id, COUNT(DISTINCT t.topic_id) as topic_count
    FROM topics t
    JOIN chapters c ON t.source_pdf = c.pdf_filename
    GROUP BY c.subject_id
    ORDER BY c.subject_id
""").fetchall()

for subject_id, count in result:
    print(f"  {subject_id:<25} {count:>3} topics")

total = conn.execute("SELECT COUNT(*) FROM topics").fetchone()[0]
print(f"\n✅ Grand total topics in DB: {total}")

# ── SAMPLE DISPLAY ─────────────────────────────────────────────────────────
print("\n── SAMPLE — Economics topics ─────────────────────────")
import pandas as pd
sample = pd.read_sql("""
    SELECT t.topic_id, t.topic_text, t.source_pdf, c.chapter_name, c.class
    FROM topics t
    JOIN chapters c ON t.source_pdf = c.pdf_filename
    WHERE c.subject_id = 'economics'
    ORDER BY c.class, c.chapter_number, t.topic_id
    LIMIT 20
""", conn)
print(sample.to_string(index=False))

Total topics to insert: 273

✅ Inserted : 273
⏭️  Skipped  : 0 (already existed)

── TOPICS BY SUBJECT ─────────────────────────────────
  business_studies           44 topics
  economics                  74 topics
  polity                     56 topics
  psychology                 49 topics
  sociology                  50 topics

✅ Grand total topics in DB: 273

── SAMPLE — Economics topics ─────────────────────────
 topic_id                                                    topic_text  source_pdf                                    chapter_name  class
        1                     Indian Economy on the Eve of Independence keec101.pdf       Indian Economy on the Eve of Independence     11
        2                   Colonial economic policies and their impact keec101.pdf       Indian Economy on the Eve of Independence     11
        3                       De-industrialisation under British rule keec101.pdf       Indian Economy on the Eve of Independence     11
        4              

## Step 7 — Verify Everything

In [10]:
import pandas as pd

# Subjects
print('── SUBJECTS ──────────────────────────')
df = pd.read_sql('SELECT subject_id, subject_name FROM subjects', conn)
print(df.to_string(index=False))

# Recommendations count
print('\n── RECOMMENDATIONS COUNT ─────────────')
df2 = pd.read_sql("""
    SELECT subject_id, level, level_type, COUNT(*) as total
    FROM recommendations
    GROUP BY subject_id, level, level_type
    ORDER BY subject_id, level, level_type
""", conn)
print(df2.to_string(index=False))
total = pd.read_sql('SELECT COUNT(*) as total FROM recommendations', conn)
print(f'\n✅ Recommendations total: {total["total"][0]}')

# Chapters count per subject
print('\n── CHAPTERS COUNT ────────────────────')
df3 = pd.read_sql("""
    SELECT subject_id, class, COUNT(*) as chapters
    FROM chapters
    GROUP BY subject_id, class
    ORDER BY subject_id, class
""", conn)
print(df3.to_string(index=False))
total_ch = pd.read_sql('SELECT COUNT(*) as total FROM chapters', conn)
print(f'\n✅ Chapters total: {total_ch["total"][0]}')

# Sample lookup — what leec102.pdf resolves to
print('\n── SAMPLE LOOKUP ─────────────────────')
sample = pd.read_sql("""
    SELECT pdf_filename, subject_id, class, chapter_number, chapter_name, book_name
    FROM chapters WHERE pdf_filename = 'leec102.pdf'
""", conn)
print(sample.to_string(index=False))

# All tables
print('\n── TABLES IN DB ──────────────────────')
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
print(tables.to_string(index=False))

── SUBJECTS ──────────────────────────
      subject_id     subject_name
       sociology        Sociology
       economics        Economics
          polity           Polity
      psychology       Psychology
business_studies Business Studies

── RECOMMENDATIONS COUNT ─────────────
      subject_id        level level_type  total
business_studies     Advanced      Paper      7
business_studies     Beginner      NCERT      4
business_studies Intermediate       Book      4
       economics     Advanced      Paper      7
       economics     Beginner      NCERT      4
       economics Intermediate       Book      4
          polity     Advanced      Paper      6
          polity     Beginner      NCERT      4
          polity Intermediate       Book      4
      psychology     Advanced      Paper      7
      psychology     Beginner      NCERT      2
      psychology Intermediate       Book      4
       sociology     Advanced      Paper      7
       sociology     Beginner      NCERT     

## ✅ Done!

Your database is ready at:
```
Google Drive/MyDrive/ncert_app/ncert.db
```

Now in `app.py`, just connect with:
```python
DB_PATH = '/content/drive/MyDrive/ncert_app/ncert.db'
conn = sqlite3.connect(DB_PATH)
```

No `init_db()` or `seed_db()` needed in `app.py` anymore.

---
## ➕ How to Add New Subjects, Books, or Chapters Later

The app reads **everything from this database** — no code changes needed ever.

### Add a new subject
```python
cur.execute("""
    INSERT OR IGNORE INTO subjects VALUES
    ('history', 'History', '📜', '#8B4513', 'Ancient, medieval and modern Indian history')
""")
conn.commit()
```

### Add books/papers for the new subject
```python
cur.execute("""
    INSERT OR IGNORE INTO recommendations
    (rec_id, subject_id, level, level_type, title, author, class, why, link)
    VALUES
    ('his_beg_1', 'history', 'Beginner', 'NCERT',
     'Themes in Indian History Part I', 'NCERT', 'Class 12',
     'Foundation for ancient and medieval India.', 'https://ncert.nic.in/...')
""")
conn.commit()
```

### Add chapters for new subject
```python
chapters = [
    ('inh101.pdf', 'class 12 history', 'history', 12, 1, 'The Story of the First Cities', 'Themes in Indian History Part I', 0),
    # ... more chapters
]
cur.executemany("""
    INSERT OR IGNORE INTO chapters
    (pdf_filename, zip_source, subject_id, class, chapter_number, chapter_name, book_name, is_special)
    VALUES (?,?,?,?,?,?,?,?)
""", chapters)
conn.commit()
```

### Add a new level (e.g. 'Expert')
Just insert recommendations with `level='Expert'` — the app sidebar picks it up automatically.

```python
cur.execute("""
    INSERT OR IGNORE INTO recommendations
    (rec_id, subject_id, level, level_type, title, author, class, why, link)
    VALUES ('eco_exp_1', 'economics', 'Expert', 'Paper',
     'Some Advanced Paper', 'Author Name', 'Reference',
     'Why read this.', 'https://link.to/paper')
""")
conn.commit()
```

> **The app sidebar will show the new subject, level, and chapters automatically on next reload. Zero code changes needed.**

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# ONE-TIME SETUP CELL — Run this ONCE to build the embedding index in DB.
# After this cell completes successfully, you NEVER need to run it again.
# The main app.py loads embeddings directly from DB — no PDFs needed.
# ─────────────────────────────────────────────────────────────────────────────

import os
import re
import hashlib
import zipfile
import sqlite3
from pathlib import Path

import numpy as np
import gdown
import requests
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer

# ─────────────────────────────────────────────
# CONFIG — update these if your paths differ
# ─────────────────────────────────────────────
FILE_ID     = "1iKxF0Db1YjySxbFVCfQhOJ6CXYVwdxbC"
ZIP_PATH    = "ncert.zip"
EXTRACT_DIR = "ncert_extracted"
DB_PATH     = "/content/drive/MyDrive/ZenithIndia/AI Learning Assistant/ncert.db"
MODEL_NAME  = "all-MiniLM-L6-v2"

# ─────────────────────────────────────────────
# STEP 0 — Mount Google Drive
# ─────────────────────────────────────────────
from google.colab import drive
if not os.path.exists("/content/drive/MyDrive"):
    drive.mount("/content/drive")
    print("✅ Drive mounted")
else:
    print("✅ Drive already mounted")

# ─────────────────────────────────────────────
# STEP 1 — Download ZIP from Google Drive
# ─────────────────────────────────────────────
def download_zip():
    if os.path.exists(ZIP_PATH) and os.path.getsize(ZIP_PATH) > 10_000:
        print(f"✅ ZIP already downloaded: {ZIP_PATH}")
        return

    print("⬇️  Downloading NCERT dataset from Google Drive...")
    downloaded = False

    # Try gdown first
    try:
        gdown.download(id=FILE_ID, output=ZIP_PATH, quiet=False, fuzzy=True)
        downloaded = os.path.exists(ZIP_PATH) and os.path.getsize(ZIP_PATH) > 10_000
        if downloaded:
            print("✅ Downloaded via gdown")
    except Exception as e:
        print(f"⚠️  gdown failed: {e} — trying fallback...")

    # Fallback: requests
    if not downloaded:
        try:
            session  = requests.Session()
            URL      = "https://drive.google.com/uc?export=download"
            response = session.get(URL, params={"id": FILE_ID}, stream=True)
            token    = next(
                (v for k, v in response.cookies.items() if k.startswith("download_warning")),
                None
            )
            if token:
                response = session.get(URL, params={"id": FILE_ID, "confirm": token}, stream=True)
            else:
                response = session.get(
                    f"https://drive.google.com/uc?id={FILE_ID}&export=download&confirm=t",
                    stream=True,
                )
            with open(ZIP_PATH, "wb") as f:
                for chunk in response.iter_content(chunk_size=32_768):
                    if chunk:
                        f.write(chunk)
            if not os.path.exists(ZIP_PATH) or os.path.getsize(ZIP_PATH) < 10_000:
                raise ValueError("Downloaded file too small — likely an error page.")
            print("✅ Downloaded via requests fallback")
        except Exception as e:
            raise RuntimeError(f"❌ Both download methods failed: {e}")

    if not zipfile.is_zipfile(ZIP_PATH):
        os.remove(ZIP_PATH)
        raise RuntimeError("❌ Downloaded file is not a valid ZIP.")

download_zip()

# ─────────────────────────────────────────────
# STEP 2 — Extract ZIPs (outer + all inner)
# ─────────────────────────────────────────────
def extract_all_zips(folder: str) -> None:
    """Recursively extract all nested ZIPs until none remain."""
    found_new = True
    while found_new:
        found_new = False
        for root, _, files in os.walk(folder):
            for file in files:
                if not file.lower().endswith(".zip"):
                    continue
                zip_path   = os.path.join(root, file)
                extract_to = os.path.join(root, os.path.splitext(file)[0])
                if os.path.exists(extract_to):
                    continue
                try:
                    with zipfile.ZipFile(zip_path, "r") as zf:
                        zf.extractall(extract_to)
                    print(f"   📂 Extracted: {file}")
                    found_new = True
                except zipfile.BadZipFile:
                    print(f"   ⚠️  Skipping corrupt ZIP: {file}")
                except Exception as e:
                    print(f"   ⚠️  Could not extract {file}: {e}")

if not os.path.exists(EXTRACT_DIR):
    print("📦 Extracting outer ZIP...")
    with zipfile.ZipFile(ZIP_PATH, "r") as zf:
        zf.extractall(EXTRACT_DIR)
    print("📂 Extracting inner subject/chapter ZIPs...")
    extract_all_zips(EXTRACT_DIR)
else:
    print(f"✅ Already extracted: {EXTRACT_DIR}")

# Count PDFs found
pdf_files = list(Path(EXTRACT_DIR).rglob("*.pdf"))
print(f"\n📄 Total PDFs found: {len(pdf_files)}")

# ─────────────────────────────────────────────
# STEP 3 — Connect to DB & ensure table exists
# ─────────────────────────────────────────────
conn = sqlite3.connect(DB_PATH)
conn.row_factory = sqlite3.Row

conn.execute("""
    CREATE TABLE IF NOT EXISTS pdf_embeddings (
        pdf_filename  TEXT PRIMARY KEY,
        embedding     BLOB NOT NULL,
        text_hash     TEXT NOT NULL,
        created_at    TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
""")
conn.commit()
print(f"✅ Connected to DB: {DB_PATH}")

# ─────────────────────────────────────────────
# STEP 4 — PDF reading utilities
# ─────────────────────────────────────────────
def read_pdf(path: str) -> str:
    try:
        reader = PdfReader(path)
        return " ".join(p.extract_text() or "" for p in reader.pages)
    except Exception as e:
        print(f"   ⚠️  Could not read {path}: {e}")
        return ""

def clean_text(text: str) -> str:
    text = re.sub(
        r"(activity|let us|exercise|project|editor|reprint|copyright|isbn).*",
        " ", text, flags=re.I
    )
    return re.sub(r"\s+", " ", text).strip()

def text_hash(text: str) -> str:
    return hashlib.md5(text.encode()).hexdigest()

# ─────────────────────────────────────────────
# STEP 5 — Load model
# ─────────────────────────────────────────────
print(f"\n🤖 Loading embedding model: {MODEL_NAME}...")
embedder = SentenceTransformer(MODEL_NAME)
print("✅ Model loaded")

# ─────────────────────────────────────────────
# STEP 6 — Read PDFs, encode, store in DB
# ─────────────────────────────────────────────
cur = conn.cursor()

# Check what's already in DB
cur.execute("SELECT pdf_filename, text_hash FROM pdf_embeddings")
already_indexed = {row["pdf_filename"]: row["text_hash"] for row in cur.fetchall()}
print(f"\n📐 Already in DB : {len(already_indexed)} embeddings")
print(f"📄 PDFs on disk  : {len(pdf_files)}")

# ── Pass 1: read all PDFs, find which need encoding ──
texts_to_encode = []
paths_to_encode = []
skipped         = 0

print("\n🔍 Scanning PDFs...")
for pdf in pdf_files:
    fname = pdf.name
    text  = clean_text(read_pdf(str(pdf)))

    if len(text.split()) < 50:
        skipped += 1
        continue

    thash = text_hash(text)

    # Skip if already in DB with same content
    if fname in already_indexed and already_indexed[fname] == thash:
        skipped += 1
        continue

    texts_to_encode.append(text)
    paths_to_encode.append(fname)

print(f"   ✅ Already up-to-date : {skipped}")
print(f"   🆕 Need encoding      : {len(texts_to_encode)}")

# ── Pass 2: encode & save only what's needed ──
if not texts_to_encode:
    print("\n🎉 All embeddings already in DB — nothing to do!")
else:
    print(f"\n🧠 Encoding {len(texts_to_encode)} PDFs (this may take a few minutes)...")
    embeddings = embedder.encode(
        texts_to_encode,
        convert_to_numpy=True,
        show_progress_bar=True,
        batch_size=16
    )

    print("💾 Saving embeddings to DB...")
    for fname, text, emb in zip(paths_to_encode, texts_to_encode, embeddings):
        conn.execute("""
            INSERT OR REPLACE INTO pdf_embeddings (pdf_filename, embedding, text_hash)
            VALUES (?, ?, ?)
        """, (fname, emb.astype(np.float32).tobytes(), text_hash(text)))

    conn.commit()
    print(f"✅ Saved {len(texts_to_encode)} embeddings to DB")

# ─────────────────────────────────────────────
# STEP 7 — Final verification
# ─────────────────────────────────────────────
cur.execute("SELECT COUNT(*) as n FROM pdf_embeddings")
total = cur.fetchone()["n"]

# Decode one to verify shape
cur.execute("SELECT pdf_filename, embedding FROM pdf_embeddings LIMIT 1")
sample = cur.fetchone()
emb_sample = np.frombuffer(sample["embedding"], dtype=np.float32)

print(f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅  INDEXING COMPLETE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Total embeddings in DB : {total}
  Embedding shape        : {emb_sample.shape}   (should be (384,))
  Embedding dtype        : {emb_sample.dtype}
  Sample file            : {sample['pdf_filename']}
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
You can now run app.py — PDFs never needed again!
""")

conn.close()